# Testing different implementations of SRT

## Packages

In [1]:
%pylab inline
from collections import Counter, defaultdict
from scipy.stats import bernoulli
from scipy.stats import binom
import numpy as np

import multiprocessing

import torch
import torch.multiprocessing as mp

import matplotlib.pyplot as plt
import math
from sympy import symbols, Eq, solve, exp, log

Populating the interactive namespace from numpy and matplotlib


## Sample Randomize and Threshold (SRT)

### Algorithm


#### SRT v1

In [2]:
#Estimation of SRT with correction

def SRT_v1(X, B, t0, t1, ps, tau):
    n = len(X)
    elements = []
    for el in range(1, B+1):
        elements.append(el)

    #Calculating true count
    cx = Counter(X)

    #Calculating true relative frequency
    phix = defaultdict(int)
    for k in cx.keys():
        phix[k] = cx[k] / n

    #Randomization
    X_rand = []
    for x in X:
        ex = [0] * (B+1)
        ex[x] = 1
        for i in range(1, len(ex)):
            if ex[i] == 1:
                ex[i] = bernoulli.rvs(t1)
            else:
                ex[i] = bernoulli.rvs(t0)
        for i in range(len(ex)):
            if ex[i] == 1:
                X_rand.append(i)

    #Sampling (the implementation is different)
    sample = []
    for a in X_rand:
        rv = bernoulli.rvs(ps)
        if rv:
            sample.append(a)
    calg = Counter(sample)

    #Thresholding
    for k in calg.keys():
        if calg[k] < tau:
            calg[k] = 0
        # correction
        if calg[k] > 0:
            calg[k] = int ((calg[k] - n * ps * t0) / (ps * (t1-t0)) )

    #Calculating estimated relative frequency (with the total size = total nonzer counts)
    phialg = defaultdict(int)
    total_app = sum(list(calg.values()))
    for k in calg.keys():
        phialg[k] = calg[k] / total_app

    return cx, phix, calg, phialg

#### SRTv2

In [3]:
import torch
from collections import Counter, defaultdict

def encode_and_randomize(x, B, t0, t1, device):
    # Create tensors directly on the appropriate device
    one_hot_vector = torch.zeros(B+1, device=device)
    one_hot_vector[x] = 1
    encoded_vector = torch.where(one_hot_vector == 0,
                                 torch.bernoulli(torch.full((B+1,), float(t0), device=device)),
                                 torch.bernoulli(torch.full((B+1,), float(t1), device=device)))
    return encoded_vector

def SRT_v2(X, B, t0, t1, ps, tau, device='cuda'):

    n = len(X)

    # Calculating true count
    cx = Counter(X)

    # Calculating true relative frequency
    phix = {k: v / n for k, v in cx.items()}

    matrix2 = torch.empty((n, B+1), device=device)

    for i, x in enumerate(X):
        encoded_vector = encode_and_randomize(x, B, t0, t1, device)
        matrix2[i] = encoded_vector

    matrix1 = torch.bernoulli(torch.full((1, n), float(ps), device=device))
    result = torch.matmul(matrix1, matrix2)

    X_rand = result.cpu().tolist()
    calg = X_rand[0]

    # Thresholding
    for k in range(len(calg)):
        calg[k] = int((calg[k] - n * ps * t0) / (ps * (t1 - t0)))
        if calg[k] < tau:
            calg[k] = 0

    # Calculating estimated relative frequency (with the total size = total non-zero counts)
    total_app = sum(calg)
    phialg = {k: calg[k] / total_app for k in range(B)}

    return cx, phix, calg, phialg

### Parameter tuning (Randomization mechanism)

In [4]:

#algorithm
#SRT parameter tuning
def solve_system_SRT(n, tau, epsilon, delta):
    A = ((log(1/delta) / (2 * n))**(1/2)).evalf()
    t_1 = 1/2

    #x = p
    #y = t0
    # Define the symbols
    x, y = symbols('x y', real=True, nonnegative=True)

    # Define the equations
    equation_1 = Eq((n - tau) * x**2 * y**2 + ((tau + 1 - n) / 2) * x**2 * y + (exp(-2 * epsilon) - 1) * tau * x * y - (tau / 4) * x**2 + (1 - exp(-2 * epsilon)) * (tau / 2) * x + (1 - exp(-2 * epsilon)) * A * n, 0)
    equation_2 = Eq(exp(-epsilon) * x * y - (1 / 2) * x + 1 - exp(-epsilon), 0)

    # Solve the system of equations for x and y
    solutions = solve((equation_1, equation_2), (x, y))

    #print("All solutions:(p, t0)=", solutions)

    #Find the solution with p and smaller t0
    #print(f"Number of solutions = {len(solutions)}")
    solutions.sort(key = lambda x:x[0])
    sol = solutions[0]
    #print(f"The solution is p={sol[0]}, t_0={sol[1]}")

    #verifying if y is also the minimum when x is the minimum
    y_solutions = [s[1] for s in solutions]
    y_solutions.sort()
    if y_solutions[0] < sol[1]:
      print(f"Warning:the solution does not have the minimum t_0")

    if sol[0] < 1:
      ep_0 = log((1-sol[1]) / sol[1])
      return sol[0], sol[1], ep_0

    x, y, ep_0 = solve_system_SRT(n, tau, 0.5, delta)
    print(f"epsilon = 0.5")
    return x, y, ep_0

def calculate_epsilon_delta(t_0, t_1, p, tau, n):
    # Calculating q
    q = (1 - p * t_1) / (1 - p * t_0)

    # Calculating denominator
    D = (1-q**2 + p * (t_0 - t_1))*(t_1 - t_0)

    # Calculating alpha from tau
    alpha = (D * tau / (n*t_0) + 1) / (1-q**2) -1

    # Calculating epsilon
    epsilon = math.log(1 / q)

    # Calculating delta
    #print(-2 * n * alpha**2 * p**2 * t_0**2)
    delta = (math.exp(-2 * n * alpha**2 * p**2 * t_0**2) )

    return epsilon, delta

In [5]:
def OUE(eps, delta, n, eps_0, tune_p = False, optional_p = 1):
    t1 = 1/2
    t0 = 1/(np.exp(eps_0 / 2) + 1)

    p = (np.exp(eps) - 1) / (np.exp(eps) * t1 - t0)
    if p > 1:
      p = min(p, 1)
      print("Warning: p > 1, the privacy guarantee eps will be smaller than the assigned.")
      print("original epsilon = {}; new epsilon = {}".format(eps, np.log((1-p*t0) / (1-p*t1))) )
    if tune_p:
      p = min([p, optional_p])
      print("Warning: p is tuned. ")
      print("original epsilon = {}; new epsilon = {}".format(eps, np.log((1-p*t0) / (1-p*t1))) )

    q = (1-p*t1) / (1 - p*t0)
    eta = (1 - q**2) - p*t1 + p*t0
    alpha = np.sqrt( np.log(delta) / (-2 * n * p**2 * t0**2))
    tau = ( (1 + alpha) * (1 - q**2) * n * p * t0 / eta - n * t0 ) / (t1 - t0)
    return t1, t0, float(p), tau

In [6]:
def DE(eps, delta, n, eps_0, B, tune_p = False, optional_p = 1):
    t1 = np.exp(eps_0) / (np.exp(eps_0) + B - 1)
    t0 = 1 / (np.exp(eps_0) + B - 1)

    p = (np.exp(eps) - 1) / (np.exp(eps) * t1 - t0)
    if p > 1:
      p = min(p, 1)
      print("Warning: p > 1, the privacy guarantee eps will be smaller than the assigned.")
      print("original epsilon = {}; new epsilon = {}".format(eps, np.log((1-p*t0) / (1-p*t1))) )
    if tune_p:
      p = min([p, optional_p])
      print("Warning: p is tuned. ")
      print("original epsilon = {}; new epsilon = {}".format(eps, np.log((1-p*t0) / (1-p*t1))) )

    q = (1-p*t1) / (1 - p*t0)
    eta = (1 - q**2) - p*t1 + p*t0
    alpha = np.sqrt( np.log(delta) / (-2 * n * p**2 * t0**2))
    tau = (1 + alpha) * (1 - q**2) * n * p * t0 / eta
    return t1, t0, p, tau

In [7]:
def SYM(eps, delta, n, eps_0, tune_p = False, optional_p = 1):
    t1 = np.exp(eps_0 / 2) / (np.exp(eps_0 / 2) + 1)
    t0 = 1 / (np.exp(eps_0 / 2) + 1)

    p = (np.exp(eps) - 1) / (np.exp(eps) * t1 - t0)
    if p > 1:
      p = min(p, 1)
      print("Warning: p > 1, the privacy guarantee eps will be smaller than the assigned.")
      print("original epsilon = {}; new epsilon = {}".format(eps, np.log((1-p*t0) / (1-p*t1))) )
    if tune_p:
      p = min([p, optional_p])
      print("Warning: p is tuned. ")
      print("original epsilon = {}; new epsilon = {}".format(eps, np.log((1-p*t0) / (1-p*t1))) )


    q = (1-p*t1) / (1 - p*t0)
    eta = (1 - q**2) - p*t1 + p*t0
    alpha = np.sqrt( np.log(delta) / (-2 * n * p**2 * t0**2))
    tau = (1 + alpha) * (1 - q**2) * n * p * t0 / eta
    return t1, t0, p, tau

In [8]:
def solve_system_SRT(n, tau, epsilon, delta):
    A = ((log(1/delta) / (2 * n))**(1/2)).evalf()
    t_1 = 1/2

    x, y = symbols('x y', real=True, nonnegative=True)

    equation_1 = Eq((n - tau) * x**2 * y**2 + ((tau + 1 - n) / 2) * x**2 * y + (exp(-2 * epsilon) - 1) * tau * x * y - (tau / 4) * x**2 + (1 - exp(-2 * epsilon)) * (tau / 2) * x + (1 - exp(-2 * epsilon)) * A * n, 0)
    equation_2 = Eq(exp(-epsilon) * x * y - (1 / 2) * x + 1 - exp(-epsilon), 0)

    solutions = solve((equation_1, equation_2), (x, y))

    solutions.sort(key = lambda x:x[0])
    sol = solutions[0]

    y_solutions = [s[1] for s in solutions]
    y_solutions.sort()
    if y_solutions[0] < sol[1]:
      print(f"Warning:the solution does not have the minimum t_0")

    if sol[0] < 1:
      ep_0 = log((1-sol[1]) / sol[1])
      return sol[0], sol[1], ep_0

    x, y, ep_0 = solve_system_SRT(n, tau, 0.5, delta)
    print(f"epsilon = 0.5")
    return x, y, ep_0

def calculate_epsilon_delta(t_0, t_1, p, tau, n):
    # Calculating q
    q = (1 - p * t_1) / (1 - p * t_0)

    # Calculating denominator
    D = (1-q**2 + p * (t_0 - t_1))*(t_1 - t_0)

    # Calculating alpha from tau
    alpha = (D * tau / (n*t_0) + 1) / (1-q**2) -1

    # Calculating epsilon
    epsilon = math.log(1 / q)

    # Calculating delta
    #print(-2 * n * alpha**2 * p**2 * t_0**2)
    delta = (math.exp(-2 * n * alpha**2 * p**2 * t_0**2) )

    return epsilon, delta

## Sample and Threshold (ST)

### Algorithm

In [9]:
#Estimation of ST with correction

def ST_sim(X, B, ps, tau):
    n = len(X)

    #counting true frequency
    cx = Counter(X)

    #obtaining true relative frequency
    phix = defaultdict(int)
    for k in cx.keys():
        phix[k] = cx[k] / n

    #sampling
    sample = []
    for a in X:
        rv = bernoulli.rvs(ps)
        if rv:
            sample.append(a)
    calg = Counter(sample)

    calg[k] = int(calg[k] / ps)  # correction
    #tau_corrected = tau / ps
    #thresholding
    for k in calg.keys():
        if calg[k] < tau:
            calg[k] = 0

    #obtaining estimated relative frequency
    phialg = defaultdict(int)
    total_app = sum(list(calg.values()))
    for k in calg.keys():
        phialg[k] = calg[k] / total_app

    return cx, phix, calg, phialg

#cx = true count
#phix = true relative frequency
#calg = estimated count after ST
#phialg = estimated relative frequency

### Parameter Tuning

In [10]:
def st_param(eps, delta, alpha = 0.2):
    if eps > 1:
      print("Warning of ST: eps is not allowed")
    ps = alpha * (1 - np.exp(-eps))
    Calp = np.log(1/alpha) - 1/(1+alpha)
    if Calp < 0:
      print("Warning of ST: C_alpha < 0")
    tau = 1 / Calp * np.log(1 / delta)
    return ps, tau / ps

## Unary Encoding (UE)

### Algorithm

In [11]:
#Algorithm of unary encoding

import torch
from collections import Counter, defaultdict

def encode_and_randomize(x, B, t0, t1, device):
    # Create tensors directly on the appropriate device
    one_hot_vector = torch.zeros(B+1, device=device)
    one_hot_vector[x] = 1
    encoded_vector = torch.where(one_hot_vector == 0,
                                 torch.bernoulli(torch.full((B+1,), float(t0), device=device)),
                                 torch.bernoulli(torch.full((B+1,), float(t1), device=device)))
    return encoded_vector

def UE(X, B, t0, t1, ps, tau, device='cuda'):
    n = len(X)

    # Calculating true count
    cx = Counter(X)

    # Calculating true relative frequency
    phix = {k: v / n for k, v in cx.items()}

    matrix2 = torch.empty((n, B+1), device=device)

    for i, x in enumerate(X):
        encoded_vector = encode_and_randomize(x, B, t0, t1, device)
        matrix2[i] = encoded_vector

    matrix1 = torch.bernoulli(torch.full((1, n), 1.0, device=device))
    result = torch.matmul(matrix1, matrix2)

    X_rand = result.cpu().tolist()
    calg = X_rand[0]

    # Thresholding
    for k in range(len(calg)):
        calg[k] = int((calg[k] - n * 1 * t0) / (1 * (t1 - t0)))


    # Calculating estimated relative frequency (with the total size = total non-zero counts)
    total_app = sum(calg)
    phialg = {k: calg[k] / total_app for k in range(B)}

    return cx, phix, calg, phialg

def UE_raw(X, B, t0, t1, ps, tau, device='cuda'):

    n = len(X)

    # Calculating true count
    cx = Counter(X)

    # Calculating true relative frequency
    phix = {k: v / n for k, v in cx.items()}

    matrix2 = torch.empty((n, B+1), device=device)

    for i, x in enumerate(X):
        encoded_vector = encode_and_randomize(x, B, t0, t1, device)
        matrix2[i] = encoded_vector

    matrix1 = torch.bernoulli(torch.full((1, n), 1.0, device=device))
    result = torch.matmul(matrix1, matrix2)

    X_rand = result.cpu().tolist()
    calg = X_rand[0]


    # Calculating estimated relative frequency (with the total size = total non-zero counts)
    total_app = sum(calg)
    phialg = {k: calg[k] / total_app for k in range(B)}

    return cx, phix, calg, phialg

### Parameter Tuning

In [12]:
#Parameter Tuning of Unary Encoding

def ue_param(eps):
  t1 = 1/2
  t0 = 1/(np.exp(eps)+1)
  return t1, t0

## Exp . Shakespeare dataset

### Experiment

#### Input data

#### break1

In [13]:
word_frequencies={'act': 313, 'i': 22891, 'scene': 787, 'rousillon': 22, 'the': 29209, 'count': 117, 's': 7884, 'palace': 156, 'enter': 2427, 'bertram': 134, 'countess': 114, 'of': 17838, 'helena': 193, 'and': 26886, 'lafeu': 116, 'all': 4071, 'in': 11578, 'black': 182, 'my': 12571, 'son': 676, 'from': 2772, 'me': 7868, 'bury': 43, 'a': 15184, 'second': 643, 'husband': 338, 'going': 109, 'madam': 520, 'weep': 180, 'o': 3143, 'er': 667, 'father': 1104, 'death': 970, 'but': 6473, 'must': 1543, 'attend': 128, 'his': 7293, 'majesty': 276, 'command': 181, 'to': 20483, 'whom': 456, 'am': 2181, 'now': 2905, 'ward': 23, 'evermore': 23, 'you': 14126, 'shall': 3690, 'find': 532, 'king': 3138, 'sir': 2989, 'he': 7109, 'that': 11617, 'so': 5216, 'generally': 12, 'is': 9546, 'at': 2612, 'times': 211, 'good': 2916, 'necessity': 40, 'hold': 471, 'virtue': 214, 'whose': 670, 'worthiness': 17, 'would': 2394, 'stir': 88, 'it': 7964, 'up': 1142, 'where': 1426, 'rather': 328, 'than': 1904, 'lack': 116, 'there': 2265, 'such': 1467, 'abundance': 10, 'what': 4953, 'hope': 373, 'hath': 1992, 'under': 301, 'practises': 14, 'time': 1118, 'with': 8062, 'finds': 49, 'no': 3894, 'other': 709, 'advantage': 74, 'process': 19, 'only': 330, 'losing': 21, 'by': 3941, 'this': 6818, 'young': 477, 'gentlewoman': 65, 'had': 1517, 'how': 2239, 'sad': 184, 'passage': 37, 'tis': 1441, 'skill': 55, 'was': 2372, 'almost': 164, 'as': 5922, 'great': 933, 'honesty': 90, 'far': 286, 'have': 6039, 'made': 858, 'nature': 398, 'immortal': 28, 'should': 1617, 'play': 382, 'for': 7918, 'work': 202, 'sake': 174, 'were': 1644, 'living': 119, 'think': 1087, 'be': 7093, 'disease': 30, 'called': 66, 'man': 2065, 'speak': 1225, 'famous': 32, 'profession': 21, 'right': 355, 'de': 256, 'excellent': 123, 'indeed': 449, 'very': 816, 'lately': 32, 'spoke': 150, 'him': 5404, 'enough': 319, 'lived': 97, 'still': 593, 'if': 3620, 'knowledge': 81, 'could': 659, 'set': 489, 'against': 593, 'mortality': 16, 'lord': 3318, 'heard': 365, 'not': 8703, 'before': 846, 'notorious': 15, 'daughter': 459, 'sole': 27, 'child': 254, 'those': 568, 'hopes': 60, 'her': 4270, 'education': 12, 'promises': 24, 'she': 2769, 'which': 2436, 'makes': 376, 'fair': 800, 'gifts': 49, 'fairer': 37, 'an': 1893, 'mind': 383, 'carries': 25, 'virtuous': 99, 'qualities': 28, 'commendations': 14, 'go': 1763, 'pity': 231, 'they': 2652, 'are': 3513, 'virtues': 54, 'traitors': 59, 'too': 1285, 'better': 597, 'their': 2248, 'goodness': 60, 'your': 6789, 'get': 318, 'tears': 321, 'best': 459, 'brine': 11, 'maiden': 50, 'can': 1259, 'season': 51, 'praise': 164, 'remembrance': 66, 'never': 1072, 'approaches': 14, 'heart': 1083, 'tyranny': 36, 'sorrows': 45, 'takes': 117, 'cheek': 87, 'more': 2364, 'lest': 149, 'thought': 392, 'affect': 26, 'sorrow': 230, 'do': 3900, 'dead': 571, 'grief': 246, 'enemy': 173, 'excess': 14, 'soon': 153, 'mortal': 106, 'desire': 251, 'holy': 204, 'wishes': 34, 'understand': 100, 'we': 3742, 'thou': 5626, 'blest': 53, 'succeed': 17, 'thy': 3993, 'manners': 71, 'shape': 88, 'blood': 703, 'contend': 18, 'empire': 16, 'thee': 3206, 'share': 28, 'love': 2143, 'trust': 179, 'few': 68, 'wrong': 285, 'none': 485, 'able': 59, 'thine': 444, 'power': 345, 'use': 329, 'keep': 505, 'friend': 427, 'own': 784, 'life': 930, 'key': 31, 'cheque': 32, 'd': 5556, 'silence': 111, 'tax': 14, 'speech': 135, 'heaven': 685, 'will': 5134, 'may': 1711, 'furnish': 34, 'prayers': 99, 'pluck': 137, 'down': 683, 'fall': 357, 'on': 3263, 'head': 542, 'farewell': 358, 'courtier': 34, 'advise': 41, 'cannot': 779, 'want': 152, 'bless': 122, 'exit': 976, 'forged': 13, 'thoughts': 254, 'servants': 93, 'comfortable': 14, 'mother': 422, 'mistress': 794, 'make': 1706, 'much': 1059, 'pretty': 127, 'lady': 1014, 'credit': 55, 'exeunt': 1061, 'these': 1357, 'grace': 607, 'shed': 56, 'like': 1933, 'forgot': 89, 'imagination': 32, 'favour': 140, 't': 1177, 'undone': 58, 'away': 873, 'twere': 126, 'one': 1888, 'bright': 74, 'particular': 55, 'star': 50, 'wed': 37, 'above': 160, 'light': 313, 'sphere': 10, 'ambition': 51, 'thus': 801, 'plagues': 13, 'itself': 256, 'hind': 10, 'lion': 122, 'die': 498, 'twas': 167, 'though': 643, 'plague': 97, 'see': 1500, 'every': 564, 'hour': 325, 'sit': 225, 'draw': 216, 'brows': 46, 'eye': 457, 'our': 3144, 'table': 57, 'capable': 16, 'line': 44, 'trick': 53, 'sweet': 800, 'gone': 477, 'fancy': 60, 'who': 1394, 'comes': 633, 'here': 2434, 'parolles': 176, 'aside': 430, 'goes': 184, 'yet': 1643, 'know': 1789, 'liar': 13, 'way': 591, 'fool': 488, 'coward': 93, 'fixed': 21, 'evils': 27, 'fit': 173, 'take': 1239, 'place': 488, 'when': 2072, 'bones': 77, 'look': 933, 'cold': 219, 'wind': 201, 'withal': 145, 'full': 446, 'oft': 151, 'wisdom': 100, 'waiting': 15, 'superfluous': 18, 'folly': 86, 'save': 214, 'queen': 1000, 'monarch': 20, 'virginity': 27, 'ay': 775, 'some': 1381, 'stain': 79, 'soldier': 286, 'let': 2417, 'ask': 191, 'question': 158, 'out': 1462, 'valiant': 158, 'defence': 37, 'weak': 122, 'unfold': 33, 'us': 1684, 'warlike': 61, 'sitting': 29, 'blow': 102, 'poor': 672, 'policy': 45, 'virgins': 10, 'might': 514, 'men': 1002, 'being': 712, 'blown': 47, 'marry': 372, 'again': 793, 'breach': 38, 'yourselves': 74, 'lose': 204, 'city': 148, 'politic': 14, 'commonwealth': 29, 'preserve': 30, 'loss': 124, 'increase': 28, 'virgin': 42, 'got': 128, 'till': 656, 'first': 1358, 'lost': 283, 'metal': 27, 'once': 439, 'ten': 154, 'found': 230, 'ever': 647, 'kept': 104, 'companion': 40, 'stand': 585, 'little': 512, 'therefore': 634, 'said': 406, 'rule': 54, 'part': 517, 'accuse': 35, 'mothers': 25, 'most': 1195, 'hangs': 41, 'himself': 484, 'murders': 16, 'buried': 58, 'limit': 19, 'desperate': 65, 'breeds': 26, 'cheese': 14, 'dies': 114, 'feeding': 13, 'stomach': 46, 'besides': 114, 'peevish': 30, 'proud': 217, 'idle': 72, 'self': 125, 'sin': 185, 'choose': 103, 'loose': 54, 'within': 505, 'year': 94, 'goodly': 82, 'principal': 11, 'worse': 169, 'liking': 32, 'ill': 285, 'ne': 252, 'likes': 27, 'commodity': 20, 'gloss': 13, 'lying': 30, 'longer': 116, 'less': 222, 'worth': 206, 'off': 519, 'while': 264, 'answer': 427, 'request': 68, 'old': 691, 'wears': 43, 'cap': 53, 'fashion': 113, 'richly': 15, 'just': 151, 'tooth': 30, 'pick': 35, 'wear': 204, 'date': 19, 'french': 190, 'looks': 222, 'eats': 19, 'anything': 10, 'master': 868, 'thousand': 338, 'loves': 197, 'phoenix': 13, 'captain': 198, 'guide': 28, 'goddess': 30, 'sovereign': 161, 'counsellor': 18, 'dear': 453, 'humble': 72, 'humility': 17, 'discord': 19, 'faith': 420, 'world': 682, 'fond': 67, 'cupid': 57, 'god': 944, 'send': 259, 'well': 2321, 'court': 274, 'learning': 33, 'wish': 258, 'wishing': 12, 'body': 299, 'felt': 41, 'born': 210, 'baser': 12, 'stars': 76, 'shut': 60, 'effects': 31, 'them': 2070, 'follow': 356, 'friends': 485, 'show': 488, 'alone': 219, 'return': 247, 'thanks': 175, 'page': 456, 'monsieur': 42, 'calls': 94, 'helen': 80, 'remember': 220, 'charitable': 18, 'mars': 45, 'especially': 19, 'why': 1509, 'wars': 139, 'needs': 144, 'backward': 22, 'fight': 284, 'running': 30, 'fear': 731, 'safety': 75, 'composition': 15, 'valour': 111, 'wing': 39, 'perfect': 61, 'instruction': 16, 'serve': 192, 'wilt': 320, 'counsel': 136, 'advice': 47, 'thrust': 55, 'upon': 1816, 'else': 424, 'diest': 21, 'ignorance': 39, 'hast': 601, 'leisure': 61, 'say': 1761, 'uses': 17, 'ourselves': 116, 'lie': 335, 'sky': 49, 'gives': 145, 'free': 203, 'scope': 25, 'doth': 992, 'pull': 17, 'slow': 58, 'designs': 13, 'dull': 113, 'high': 279, 'feed': 89, 'mine': 1179, 'space': 31, 'fortune': 367, 'brings': 52, 'join': 87, 'kiss': 230, 'native': 42, 'things': 334, 'impossible': 41, 'strange': 257, 'weigh': 58, 'pains': 93, 'sense': 129, 'suppose': 33, 'been': 736, 'merit': 52, 'did': 1722, 'miss': 33, 'project': 12, 'deceive': 26, 'intents': 22, 'fix': 27, 'leave': 677, 'ii': 319, 'paris': 157, 'flourish': 128, 'cornets': 14, 'france': 414, 'letters': 132, 'divers': 24, 'attendants': 126, 'ears': 183, 'fought': 69, 'equal': 52, 'continue': 32, 'war': 272, 'reported': 14, 'nay': 599, 'received': 85, 'vouch': 16, 'cousin': 270, 'austria': 25, 'florentine': 12, 'move': 101, 'speedy': 21, 'aid': 62, 'wherein': 144, 'dearest': 53, 'business': 235, 'seem': 236, 'denial': 11, 'approved': 19, 'plead': 49, 'arm': 191, 'florence': 22, 'denied': 56, 'gentlemen': 213, 'mean': 311, 'service': 223, 'freely': 57, 'either': 179, 'gentry': 16, 'sick': 184, 'breathing': 26, 'exploit': 17, 'youth': 282, 'bear': 578, 'st': 756, 'face': 458, 'frank': 12, 'curious': 18, 'haste': 173, 'composed': 11, 'moral': 28, 'parts': 110, 'mayst': 62, 'inherit': 15, 'welcome': 376, 'duty': 159, 'corporal': 22, 'myself': 589, 'friendship': 50, 'tried': 22, 'into': 657, 'long': 504, 'both': 656, 'age': 200, 'steal': 82, 'wore': 28, 'talk': 233, 'wit': 278, 'observe': 34, 'day': 977, 'lords': 455, 'jest': 106, 'scorn': 116, 'ere': 391, 'hide': 110, 'honour': 727, 'contempt': 49, 'nor': 982, 'pride': 115, 'or': 2487, 'awaked': 16, 'clock': 84, 'knew': 166, 'true': 819, 'minute': 47, 'bid': 350, 'tongue': 433, 'obey': 90, 'hand': 885, 'below': 59, 'used': 74, 'creatures': 44, 'another': 465, 'bow': 75, 'top': 61, 'low': 111, 'ranks': 20, 'making': 81, 'copy': 12, 'younger': 35, 'lies': 261, 'richer': 19, 'tomb': 57, 'lives': 209, 'epitaph': 13, 'royal': 222, 'always': 63, 'methinks': 146, 'hear': 914, 'words': 439, 'scatter': 22, 'grow': 138, 'live': 523, 'melancholy': 74, 'began': 45, 'heel': 17, 'pastime': 14, 'quoth': 110, 'after': 430, 'flame': 24, 'lacks': 16, 'oil': 15, 'spirits': 125, 'senses': 40, 'new': 254, 'disdain': 50, 'judgments': 13, 'mere': 57, 'fathers': 46, 'garments': 46, 'since': 456, 'wax': 33, 'honey': 53, 'bring': 462, 'home': 354, 'quickly': 257, 'give': 1379, 'room': 145, 'loved': 189, 'least': 103, 'lend': 104, 'fill': 95, 'physician': 26, 'died': 114, 'famed': 12, 'six': 61, 'months': 41, 'try': 90, 'rest': 366, 'worn': 42, 'several': 83, 'sickness': 55, 'debate': 16, 'dearer': 24, 'thank': 306, 'iii': 335, 'steward': 33, 'clown': 349, 'care': 229, 'even': 625, 'content': 166, 'past': 151, 'then': 2259, 'wound': 83, 'modesty': 52, 'foul': 258, 'publish': 13, 'does': 334, 'knave': 176, 'sirrah': 149, 'complaints': 10, 'believe': 244, 'commit': 41, 'ability': 10, 'yours': 262, 'unknown': 46, 'fellow': 329, 'many': 590, 'rich': 173, 'damned': 75, 'ladyship': 44, 'woman': 419, 'beggar': 80, 'beg': 101, 'case': 111, 'blessing': 48, 'issue': 113, 'blessings': 19, 'tell': 1103, 'reason': 303, 'requires': 16, 'driven': 19, 'flesh': 147, 'devil': 245, 'drives': 14, 'worship': 105, 'reasons': 63, 'wicked': 71, 'creature': 79, 'repent': 59, 'marriage': 124, 'sooner': 61, 'wickedness': 11, 'wife': 521, 'enemies': 112, 're': 428, 'shallow': 256, 'knaves': 44, 'come': 2584, 'land': 225, 'crop': 12, 'cuckold': 35, 'comforts': 23, 'kisses': 43, 'contented': 26, 'hearts': 175, 'religion': 18, 'heads': 112, 'horns': 54, 'together': 267, 'any': 835, 'deer': 44, 'herd': 18, 'prophet': 16, 'truth': 331, 'next': 170, 'ballad': 19, 'repeat': 17, 'destiny': 22, 'cuckoo': 22, 'sings': 71, 'kind': 288, 'll': 2480, 'anon': 131, 'please': 400, 'cause': 350, 'troy': 96, 'done': 702, 'priam': 52, 'joy': 208, 'stood': 107, 'gave': 246, 'sentence': 43, 'among': 92, 'nine': 61, 'bad': 131, 'corrupt': 29, 'song': 87, 'ld': 87, 'fault': 180, 'parson': 21, 'twould': 28, 'mend': 58, 'hurt': 97, 'over': 232, 'gown': 57, 'big': 46, 'forsooth': 48, 'hither': 308, 'herself': 113, 'without': 377, 'title': 99, 'paid': 65, 'demand': 64, 'late': 208, 'near': 265, 'wished': 14, 'dare': 209, 'vow': 102, 'stranger': 60, 'matter': 363, 'put': 508, 'difference': 39, 'betwixt': 60, 'two': 698, 'extend': 17, 'level': 24, 'dian': 27, 'suffer': 108, 'knight': 156, 'surprised': 22, 'rescue': 30, 'assault': 20, 'ransom': 55, 'delivered': 30, 'bitter': 78, 'touch': 169, 'e': 128, 'exclaim': 12, 'held': 87, 'speedily': 12, 'acquaint': 22, 'happen': 10, 'concerns': 21, 'something': 183, 'discharged': 20, 'yourself': 283, 'hung': 24, 'balance': 11, 'neither': 192, 'pray': 775, 'bosom': 135, 'honest': 298, 'further': 209, 'ours': 94, 'thorn': 13, 'rose': 70, 'rightly': 27, 'belong': 19, 'seal': 101, 'strong': 206, 'passion': 120, 'impress': 10, 'days': 206, 'faults': 95, 'pleasure': 181, 'honourable': 127, 'methought': 46, 'saw': 284, 'serpent': 34, 'start': 35, 'often': 126, 'seen': 273, 'strives': 12, 'choice': 75, 'slip': 23, 'foreign': 29, 'oppress': 14, 'groan': 39, 'express': 52, 'mercy': 184, 'distemper': 18, 'messenger': 320, 'wet': 23, 'colour': 101, 'pardon': 315, 'brother': 626, 'name': 688, 'note': 159, 'parents': 19, 'noble': 636, 'servant': 392, 'vassal': 19, 'sister': 194, 'yes': 210, 'law': 232, 'shield': 28, 'strive': 37, 'pulse': 10, 'pale': 156, 'catch': 73, 'mystery': 18, 'salt': 53, 'gross': 59, 'invention': 27, 'ashamed': 36, 'proclamation': 22, 'dost': 419, 'cheeks': 89, 'confess': 144, 'th': 32, 'eyes': 730, 'grossly': 19, 'shown': 33, 'tie': 22, 'forswear': 30, 'howe': 12, 'charge': 204, 'truly': 169, 'about': 415, 'bond': 74, 'whereof': 84, 'state': 312, 'affection': 83, 'passions': 19, 'knee': 66, 'unto': 460, 'offended': 47, 'hurts': 13, 'token': 28, 'suit': 151, 'deserve': 69, 'desert': 47, 'vain': 96, 'pour': 44, 'waters': 31, 'religious': 20, 'error': 34, 'adore': 14, 'sun': 254, 'knows': 207, 'hate': 169, 'encounter': 64, 'loving': 113, 'aged': 31, 'dearly': 43, 'sure': 300, 'seeks': 32, 'search': 54, 'riddle': 10, 'sweetly': 28, 'intent': 54, 'wherefore': 139, 'swear': 266, 'left': 262, 'rare': 58, 'proved': 41, 'reading': 37, 'manifest': 13, 'experience': 22, 'collected': 13, 'general': 210, 'sovereignty': 27, 'bestow': 69, 'notes': 22, 'amongst': 43, 'remedy': 58, 'cure': 57, 'render': 53, 'motive': 17, 'medicine': 25, 'conversation': 11, 'haply': 44, 'absent': 30, 'tender': 157, 'supposed': 34, 'receive': 98, 'help': 333, 'danger': 111, 'greatest': 55, 'receipt': 12, 'success': 49, 'venture': 29, 'shalt': 304, 'means': 268, 'stay': 462, 'attempt': 37, 'morrow': 345, 'attended': 47, 'taking': 47, 'throw': 105, 'gain': 74, 'gift': 62, 'stretch': 29, 'soldiers': 194, 'health': 104, 'owes': 26, 'malady': 11, 'whether': 110, 'sons': 154, 'worthy': 231, 'frenchmen': 19, 'higher': 37, 'italy': 35, 'last': 299, 'woo': 111, 'seek': 242, 'fame': 82, 'cry': 228, 'loud': 71, 'bidding': 28, 'girls': 13, 'heed': 52, 'language': 40, 'deny': 138, 'beware': 30, 'behind': 129, 'spark': 10, 'brave': 161, 'admirable': 15, 'commanded': 39, 'coil': 12, 'early': 55, 'boy': 428, 'bravely': 35, 'shoes': 20, 'plain': 120, 'bought': 54, 'sword': 355, 'dance': 78, 'theft': 16, 'parting': 30, 'kin': 35, 'sparks': 10, 'word': 518, 'reports': 19, 'dote': 30, 'ye': 315, 'retire': 46, 'spacious': 12, 'ceremony': 27, 'list': 59, 'adieu': 101, 'themselves': 168, 'muster': 23, 'gait': 35, 'eat': 149, 'lead': 149, 'measure': 97, 'followed': 19, 'fellows': 75, 'prove': 252, 'kneeling': 21, 'tidings': 55, 'fee': 41, 'stands': 162, 'has': 396, 'brought': 212, 'kneel': 74, 'broke': 86, 'pate': 31, 'cured': 13, 'infirmity': 17, 'fox': 37, 'reach': 30, 'breathe': 63, 'stone': 77, 'quicken': 12, 'rock': 42, 'fire': 300, 'motion': 79, 'simple': 114, 'powerful': 19, 'pen': 33, 'write': 115, 'doctor': 180, 'arrived': 28, 'convey': 52, 'sex': 19, 'years': 199, 'constancy': 14, 'amazed': 44, 'blame': 91, 'weakness': 26, 'laugh': 106, 'admiration': 14, 'spend': 68, 'wonder': 139, 'took': 171, 'special': 37, 'nothing': 640, 'ways': 80, 'wings': 61, 'traitor': 141, 'seldom': 32, 'fears': 76, 'cressid': 36, 'uncle': 198, 'fare': 138, 'profess': 31, 'spare': 63, 'praises': 28, 'towards': 76, 'knowing': 49, 'bed': 340, 'chiefly': 14, 'practise': 63, 'bade': 50, 'store': 38, 'safer': 19, 'hearing': 88, 'chief': 101, 'bound': 187, 'credulous': 11, 'learned': 56, 'concluded': 17, 'labouring': 17, 'art': 911, 'estate': 54, 'judgment': 109, 'esteem': 35, 'senseless': 29, 'deem': 11, 'pay': 148, 'enforce': 34, 'office': 132, 'humbly': 61, 'modest': 48, 'back': 358, 'call': 738, 'peril': 43, 'gainst': 101, 'works': 24, 'minister': 45, 'writ': 105, 'babes': 24, 'floods': 11, 'seas': 48, 'dried': 16, 'expectation': 25, 'despair': 57, 'fits': 47, 'maid': 205, 'thyself': 218, 'reap': 16, 'reward': 39, 'breath': 228, 'barr': 16, 'square': 14, 'guess': 53, 'shows': 108, 'consent': 100, 'proclaim': 62, 'aim': 54, 'confident': 19, 'twice': 74, 'horses': 63, 'fiery': 50, 'ring': 169, 'moist': 12, 'quench': 36, 'sleepy': 11, 'lamp': 10, 'four': 151, 'twenty': 168, 'pilot': 11, 'glass': 68, 'told': 241, 'minutes': 13, 'pass': 180, 'sound': 215, 'fly': 262, 'confidence': 16, 'darest': 51, 'strumpet': 29, 'boldness': 15, 'shame': 346, 'odious': 10, 'otherwise': 37, 'torture': 31, 'ended': 33, 'blessed': 78, 'spirit': 257, 'slay': 32, 'common': 157, 'rate': 33, 'beauty': 211, 'courage': 74, 'happiness': 64, 'prime': 18, 'happy': 209, 'hazard': 43, 'infinite': 38, 'monstrous': 65, 'physic': 36, 'ministers': 23, 'break': 268, 'property': 15, 'deserved': 59, 'promise': 111, 'sceptre': 33, 'kingly': 28, 'forth': 390, 'branch': 17, 'image': 51, 'observed': 16, 'performance': 18, 'served': 55, 'resolved': 50, 'patient': 92, 'whence': 90, 'camest': 30, 'ho': 214, 'proceed': 75, 'deed': 169, 'match': 95, 'meed': 19, 'height': 35, 'breeding': 32, 'highly': 16, 'fed': 42, 'lowly': 14, 'taught': 50, 'lent': 38, 'easily': 31, 'leg': 39, 'hands': 325, 'lip': 37, 'questions': 13, 'barber': 10, 'chair': 39, 'pin': 21, 'crown': 306, 'rush': 26, 'tom': 30, 'nail': 12, 'hole': 33, 'horn': 46, 'friar': 195, 'mouth': 145, 'skin': 36, 'duke': 1144, 'beneath': 23, 'constable': 81, 'size': 12, 'demands': 19, 'trifle': 24, 'belongs': 15, 'harm': 103, 'learn': 121, 'wiser': 15, 'putting': 25, 'hundred': 114, 'thick': 46, 'homely': 16, 'meat': 73, 'warrant': 188, 'whipped': 26, 'whipping': 10, 'luck': 25, 'housewife': 11, 'entertain': 55, 'merrily': 23, 'serves': 38, 'end': 335, 'urge': 34, 'present': 252, 'commend': 96, 'kinsmen': 18, 'commendation': 12, 'employment': 21, 'legs': 74, 'severally': 28, 'persons': 21, 'familiar': 37, 'hence': 369, 'trifles': 16, 'seeming': 64, 'submit': 12, 'rarest': 11, 'argument': 63, 'shot': 52, 'latter': 19, 'assured': 52, 'uncertain': 10, 'showing': 15, 'read': 209, 'heavenly': 60, 'effect': 73, 'earthly': 35, 'actor': 13, 'same': 324, 'fore': 44, 'respect': 106, 'brief': 93, 'tedious': 50, 'acknowledge': 18, 'recovery': 11, 'thankful': 21, 'says': 243, 'whilst': 109, 'side': 197, 'banish': 115, 'repeal': 15, 'promised': 76, 'attends': 21, 'three': 407, 'youthful': 30, 'parcel': 15, 'voice': 168, 'election': 22, 'forsake': 25, 'each': 255, 'bay': 31, 'broken': 62, 'boys': 86, 'beard': 94, 'peruse': 15, 'through': 297, 'restored': 17, 'therein': 60, 'protest': 55, 'simply': 11, 'already': 149, 'blushes': 14, 'whisper': 34, 'blush': 59, 'shouldst': 74, 'refused': 12, 'white': 165, 'altar': 14, 'imperial': 26, 'sighs': 63, 'stream': 35, 'grant': 112, 'mute': 17, 'flames': 13, 'replies': 10, 'fortunes': 123, 'turk': 18, 'afraid': 44, 'vows': 68, 'ice': 18, 'bastards': 17, 'english': 156, 'em': 174, 'fourth': 52, 'drunk': 68, 'wine': 85, 'ass': 92, 'fourteen': 24, 'known': 196, 'liege': 147, 'beseech': 234, 'highness': 189, 'raised': 30, 'sickly': 18, 'follows': 67, 'build': 26, 'bloods': 16, 'weight': 45, 'heat': 76, 'quite': 83, 'confound': 38, 'distinction': 10, 'mighty': 115, 'swell': 25, 'wise': 171, 'immediate': 15, 'heir': 112, 'breed': 54, 'sire': 22, 'honours': 77, 'thrive': 59, 'acts': 26, 'derive': 10, 'slave': 133, 'grave': 204, 'dumb': 49, 'dust': 64, 'damn': 56, 'oblivion': 17, 'canst': 178, 'create': 22, 'dower': 17, 'wealth': 70, 'm': 33, 'glad': 132, 'stake': 18, 'defeat': 13, 'produce': 17, 'unworthy': 38, 'vile': 122, 'dream': 144, 'scale': 15, 'beam': 11, 'plant': 23, 'presently': 147, 'obedient': 30, 'careless': 17, 'revenge': 160, 'justice': 240, 'terms': 77, 'gracious': 198, 'consider': 60, 'dole': 10, 'flies': 61, 'nobler': 20, 'base': 168, 'praised': 43, 'smile': 81, 'contract': 27, 'perform': 81, 'night': 917, 'solemn': 43, 'feast': 108, 'coming': 160, 'lovest': 29, 'err': 17, 'advancing': 10, 'harsh': 30, 'understood': 13, 'bloody': 235, 'succeeding': 10, 'style': 18, 'satisfy': 29, 'didst': 187, 'vent': 20, 'travel': 34, 'vessel': 38, 'burthen': 30, 'scarce': 78, 'hadst': 106, 'privilege': 32, 'anger': 66, 'trial': 61, 'window': 46, 'need': 186, 'open': 177, 'dram': 16, 'bate': 17, 'scruple': 23, 'contrary': 49, 'scarf': 11, 'beaten': 51, 'bondage': 22, 'acquaintance': 37, 'vexation': 11, 'hell': 177, 'doing': 77, 'eternal': 36, 'disgrace': 48, 'scurvy': 27, 'filthy': 21, 'authority': 57, 'beat': 134, 'meet': 328, 'double': 92, 'married': 150, 'news': 321, 'lordship': 130, 'wrongs': 71, 'garter': 32, 'arms': 282, 'hose': 23, 'sleeves': 10, 'wert': 105, 'lower': 30, 'nose': 71, 'hours': 137, 'offence': 117, 'wast': 78, 'created': 16, 'hard': 196, 'traveller': 15, 'saucy': 33, 'commission': 53, 'birth': 95, 'awhile': 106, 'cares': 45, 'although': 79, 'priest': 68, 'sworn': 135, 'dog': 158, 'merits': 13, 'tread': 47, 'foot': 190, 'import': 19, 'box': 25, 'unseen': 23, 'manly': 22, 'sustain': 19, 'steed': 21, 'dwell': 39, 'jades': 12, 'house': 580, 'fled': 91, 'durst': 54, 'italian': 14, 'fields': 33, 'strike': 161, 'strife': 41, 'dark': 90, 'detested': 16, 'chamber': 144, 'straight': 158, 'single': 55, 'balls': 18, 'noise': 111, 'marr': 12, 'hush': 22, 'iv': 336, 'greets': 13, 'kindly': 37, 'merry': 176, 'given': 195, 'wants': 41, 'whither': 92, 'earth': 338, 'fortunate': 19, 'wrinkles': 11, 'money': 183, 'shakes': 21, 'rt': 43, 'witty': 16, 'laughter': 28, 'serious': 25, 'due': 76, 'puts': 49, 'compell': 16, 'restraint': 13, 'delay': 40, 'strew': 25, 'sweets': 12, 'drown': 79, 'instant': 60, 'proceeding': 26, 'strengthen': 12, 'probable': 10, 'commands': 30, 'having': 171, 'obtain': 17, 'thing': 508, 'wait': 65, 'report': 148, 'v': 258, 'thinks': 64, 'testimony': 13, 'dial': 10, 'lark': 24, 'assure': 51, 'dangerous': 106, 'pursue': 35, 'amity': 24, 'tailor': 49, 'treasure': 54, 'order': 99, 'possession': 34, 'bride': 39, 'begin': 124, 'dinner': 96, 'thrice': 83, 'unkindness': 22, 'between': 244, 'run': 201, 'displeasure': 43, 'shift': 28, 'boots': 27, 'spurs': 16, 'nut': 10, 'soul': 473, 'clothes': 42, 'heavy': 181, 'consequence': 19, 'tame': 39, 'natures': 25, 'spoken': 60, 'evil': 64, 'desires': 84, 'private': 76, 'marvel': 36, 'course': 167, 'holds': 82, 'required': 12, 'prepared': 41, 'entreat': 115, 'muse': 15, 'respects': 20, 'greater': 94, 'view': 87, 'giving': 65, 'letter': 246, 'twill': 66, 'observance': 16, 'toward': 97, 'fail': 67, 'hie': 50, 'owe': 80, 'thief': 69, 'fain': 67, 'strangers': 20, 'foes': 74, 'sunder': 13, 'horse': 255, 'shake': 102, 'drum': 112, 'flight': 57, 'troop': 30, 'point': 152, 'seems': 127, 'quarrel': 98, 'fearful': 97, 'yield': 157, 'outward': 51, 'figure': 41, 'council': 49, 'surfeit': 19, 'ease': 61, 'places': 43, 'fell': 135, 'field': 203, 'along': 117, 'troth': 106, 'boot': 31, 'sing': 143, 'teeth': 60, 'sold': 39, 'writes': 25, 'opening': 12, 'country': 186, 'brains': 46, 'en': 149, 'reads': 104, 'sent': 271, 'recovered': 13, 'wedded': 18, 'breadth': 10, 'distance': 25, 'unfortunate': 12, 'rash': 39, 'favours': 41, 'indignation': 11, 'yonder': 75, 'comfort': 197, 'killed': 40, 'standing': 42, 'getting': 11, 'children': 129, 'gentleman': 575, 'patience': 200, 'met': 141, 'thence': 94, 'came': 360, 'dispatch': 89, 'thither': 92, 'bend': 48, 'finger': 69, 'dreadful': 65, 'contents': 27, 'sorry': 92, 'pain': 75, 'prithee': 215, 'cheer': 82, 'griefs': 61, 'robb': 22, 'moiety': 16, 'wash': 77, 'purpose': 214, 'lay': 299, 'speed': 244, 'until': 80, 'deserves': 37, 'rude': 72, 'tend': 23, 'hourly': 24, 'sometime': 82, 'tainted': 19, 'derived': 20, 'deal': 55, 'win': 121, 'written': 32, 'worthiest': 13, 'affairs': 66, 'change': 156, 'courtesies': 12, 'chase': 37, 'limbs': 47, 'event': 28, 'drive': 39, 'mark': 507, 'leaden': 18, 'messengers': 19, 'ride': 55, 'violent': 43, 'false': 305, 'air': 205, 'piercing': 15, 'whoever': 13, 'charges': 14, 'forward': 92, 'breast': 111, 'caitiff': 13, 'kill': 312, 'roar': 38, 'sharp': 82, 'hunger': 19, 'miseries': 24, 'scar': 12, 'paradise': 12, 'fan': 18, 'angels': 37, 'pitiful': 32, 'rumour': 21, 'ear': 220, 'trumpets': 80, 'strength': 123, 'extreme': 16, 'edge': 41, 'prosperous': 27, 'helm': 19, 'file': 15, 'lover': 66, 'alas': 233, 'sending': 11, 'saint': 120, 'jaques': 85, 'pilgrim': 10, 'ambitious': 38, 'ground': 176, 'peace': 551, 'taken': 93, 'labours': 22, 'forgive': 75, 'juno': 24, 'dogs': 49, 'heels': 74, 'embrace': 76, 'ah': 167, 'prevented': 21, 'pursuit': 17, 'angel': 52, 'unless': 157, 'delights': 18, 'wrath': 62, 'feel': 112, 'convenient': 19, 'led': 75, 'pure': 93, 'provide': 30, 'bids': 67, 'walls': 82, 'tucket': 10, 'afar': 20, 'widow': 111, 'diana': 80, 'mariana': 47, 'citizens': 96, 'approach': 76, 'sight': 204, 'commander': 13, 'slew': 42, 'labour': 114, 'hark': 157, 'suffice': 17, 'earl': 161, 'neighbour': 57, 'hang': 199, 'officer': 93, 'oaths': 64, 'tokens': 15, 'lust': 80, 'misery': 44, 'example': 31, 'terrible': 28, 'wreck': 41, 'succession': 16, 'limed': 10, 'threaten': 17, 'disguised': 39, 'le': 51, 'grand': 14, 'lodge': 25, 'francis': 70, 'beside': 35, 'port': 18, 'march': 132, 'tarry': 45, 'troops': 30, 'conduct': 50, 'lodged': 16, 'hostess': 81, 'ample': 16, 'countryman': 19, 'hears': 34, 'nobly': 38, 'stole': 24, 'surely': 41, 'deserving': 20, 'reserved': 16, 'become': 147, 'sadly': 23, 'shrewd': 27, 'turn': 349, 'pleased': 84, 'amorous': 23, 'unlawful': 11, 'keeps': 73, 'guard': 133, 'gods': 333, 'forbid': 73, 'colours': 71, 'whole': 130, 'army': 94, 'antonio': 267, 'eldest': 27, 'escalus': 95, 'frenchman': 22, 'gallant': 58, 'handsome': 14, 'yond': 40, 'leads': 31, 'poison': 106, 'rascal': 61, 'jack': 89, 'apes': 15, 'perchance': 51, 'battle': 120, 'shrewdly': 10, 'spied': 11, 'courtesy': 77, 'carrier': 23, 'host': 146, 'five': 140, 'gentle': 380, 'requite': 25, 'offer': 119, 'vi': 249, 'camp': 74, 'deceived': 56, 'direct': 35, 'malice': 76, 'kinsman': 49, 'notable': 14, 'owner': 15, 'quality': 58, 'entertainment': 39, 'trusty': 18, 'main': 49, 'action': 119, 'fetch': 129, 'undertake': 47, 'suddenly': 53, 'surprise': 12, 'bind': 44, 'carried': 31, 'adversaries': 11, 'tents': 21, 'highest': 15, 'compulsion': 10, 'betray': 62, 'deliver': 149, 'intelligence': 26, 'divine': 51, 'forfeit': 27, 'oath': 156, 'stratagem': 10, 'sees': 54, 'bottom': 107, 'counterfeit': 55, 'melted': 16, 'john': 507, 'removed': 29, 'hinder': 19, 'design': 20, 'sticks': 13, 'disposition': 48, 'pox': 23, 'rend': 16, 'caesar': 601, 'greatly': 12, 'condemn': 45, 'dishonour': 67, 'exact': 10, 'hic': 12, 'instrument': 41, 'quarter': 28, 'enterprise': 34, 'becomes': 67, 'greatness': 80, 'utmost': 19, 'syllable': 12, 'slumber': 22, 'evening': 20, 'preparation': 28, 'midnight': 43, 'bold': 144, 'subscribe': 15, 'fish': 53, 'water': 160, 'dares': 48, 'certain': 184, 'week': 29, 'escape': 19, 'address': 20, 'clap': 15, 'sport': 127, 'disguise': 21, 'parted': 49, 'caught': 33, 'wondrous': 36, 'coxcomb': 20, 'vii': 24, 'fallen': 21, 'acquainted': 40, 'reputation': 52, 'borrow': 48, 'purse': 90, 'gold': 235, 'buy': 97, 'friendly': 39, 'lays': 23, 'wanton': 65, 'siege': 33, 'carry': 91, 'fine': 96, 'nought': 60, 'county': 28, 'lawful': 63, 'won': 98, 'delivers': 10, 'add': 46, 'crowns': 63, 'passed': 11, 'yielded': 27, 'instruct': 26, 'deceit': 22, 'sorts': 14, 'songs': 14, 'chide': 48, 'assay': 18, 'plot': 54, 'meaning': 65, 'sinful': 10, 'fact': 14, 'hedge': 19, 'corner': 21, 'band': 29, 'couch': 24, 'beguile': 24, 'sleep': 271, 'smoke': 25, 'door': 142, 'daring': 16, 'guilty': 82, 'ignorant': 48, 'slight': 30, 'ones': 64, 'instance': 26, 'butter': 11, 'possible': 58, 'breaking': 29, 'afford': 21, 'hardly': 41, 'swore': 52, 'deep': 166, 'thirty': 29, 'fathom': 11, 'believed': 18, 'alarum': 87, 'seize': 35, 'german': 12, 'discover': 42, 'undo': 32, 'gather': 32, 'inform': 40, 'secrets': 26, 'force': 115, 'purposes': 38, 'faithfully': 11, 'granted': 30, 'guarded': 34, 'short': 112, 'muffled': 10, 'safely': 26, 'lock': 48, 'addition': 26, 'frame': 46, 'quick': 112, 'monument': 39, 'rights': 23, 'roses': 28, 'thorns': 13, 'prick': 44, 'mock': 114, 'witness': 131, 'holding': 24, 'conditions': 28, 'opinion': 91, 'cruel': 69, 'integrity': 19, 'recover': 35, 'begins': 63, 'longing': 18, 'ancestors': 22, 'chastity': 27, 'jewel': 62, 'proper': 66, 'champion': 15, 'yea': 203, 'knock': 73, 'conquer': 34, 'remain': 74, 'future': 13, 'deeds': 109, 'wooing': 27, 'sat': 32, 'cozen': 11, 'changed': 45, 'laid': 96, 'shaking': 14, 'everlasting': 23, 'tuned': 12, 'bounty': 42, 'chaste': 47, 'renown': 30, 'spoil': 35, 'rebellion': 32, 'merely': 28, 'treasons': 19, 'attain': 14, 'abhorred': 11, 'ends': 51, 'nobility': 37, 'meant': 53, 'damnable': 11, 'company': 208, 'apace': 28, 'gladly': 15, 'meddle': 20, 'presence': 107, 'whip': 37, 'perceive': 94, 'altogether': 25, 'pretence': 10, 'pilgrimage': 16, 'tenderness': 13, 'became': 24, 'prey': 47, 'stronger': 39, 'story': 71, 'verity': 10, 'heartily': 40, 'mightily': 16, 'sometimes': 68, 'losses': 14, 'dignity': 38, 'web': 12, 'mingled': 16, 'crimes': 17, 'street': 83, 'morning': 137, 'offered': 14, 'needful': 31, 'sixteen': 13, 'month': 42, 'length': 39, 'piece': 101, 'nearest': 12, 'departure': 21, 'fearing': 30, 'hereafter': 52, 'stocks': 28, 'usurping': 15, 'weeps': 41, 'wench': 63, 'milk': 32, 'setting': 25, 'confession': 18, 'pinch': 41, 'merciful': 20, 'rogues': 21, 'saving': 16, 'phrase': 28, 'knot': 30, 'dagger': 43, 'keeping': 27, 'clean': 24, 'wearing': 17, 'apparel': 39, 'marvellous': 22, 'fifty': 33, 'sebastian': 132, 'rotten': 39, 'fifteen': 15, 'half': 245, 'snow': 45, 'pieces': 40, 'condition': 62, 'dumain': 75, 'sums': 16, 'revolt': 42, 'innocent': 53, 'falls': 83, 'pocket': 34, 'sadness': 25, 'tent': 99, 'paper': 103, 'foolish': 98, 'behalf': 45, 'lascivious': 13, 'sides': 48, 'rogue': 81, 'swears': 29, 'drop': 81, 'pays': 26, 'score': 36, 'debts': 18, 'rhyme': 30, 'forehead': 24, 'endure': 77, 'cat': 40, 'offences': 26, 'remainder': 11, 'answered': 20, 'egg': 18, 'hercules': 36, 'swine': 13, 'straw': 19, 'description': 19, 'belie': 10, 'except': 47, 'mile': 34, 'villany': 51, 'price': 31, 'sell': 40, 'inheritance': 12, 'cut': 182, 'perpetual': 24, 'crow': 33, 'nest': 26, 'reputed': 10, 'retreat': 26, 'lackey': 10, 'saved': 39, 'drums': 42, 'discovered': 11, 'greeting': 24, 'compel': 10, 'women': 187, 'nation': 32, 'burst': 29, 'drink': 161, 'soft': 136, 'rust': 12, 'cool': 37, 'foolery': 17, 'alive': 96, 'christian': 73, 'surety': 21, 'throne': 68, 'desired': 31, 'flinty': 12, 'peep': 22, 'recompense': 27, 'doubt': 183, 'summer': 75, 'leaves': 70, 'whate': 27, 'misled': 10, 'villanous': 26, 'advanced': 19, 'red': 106, 'bee': 14, 'cost': 46, 'groans': 43, 'owed': 12, 'rooted': 10, 'herb': 10, 'herbs': 12, 'grass': 29, 'prince': 736, 'darkness': 45, 'narrow': 21, 'gate': 68, 'pomp': 33, 'broad': 20, 'because': 193, 'looked': 32, 'tricks': 34, 'unhappy': 46, 'remains': 44, 'pace': 32, 'runs': 50, 'amiss': 37, 'moved': 83, 'stop': 103, 'conceived': 10, 'fitter': 11, 'happily': 28, 'post': 88, 'thinking': 46, 'admitted': 13, 'patch': 18, 'velvet': 18, 'pile': 11, 'bare': 65, 'livery': 27, 'belike': 43, 'dozen': 35, 'delicate': 30, 'courteous': 22, 'feathers': 14, 'nod': 19, 'exceeding': 26, 'nights': 38, 'presume': 21, 'occasions': 16, 'nice': 32, 'petition': 21, 'adverse': 11, 'unfit': 11, 'following': 65, 'mood': 22, 'smell': 53, 'somewhat': 20, 'strongly': 28, 'henceforth': 43, 'allow': 40, 'spake': 54, 'foh': 12, 'close': 112, 'stool': 17, 'nobleman': 17, 'rascally': 11, 'distress': 24, 'nails': 32, 'played': 15, 'scratch': 25, 'justices': 12, 'penny': 25, 'ha': 240, 'sooth': 46, 'inquire': 24, 'mad': 260, 'estimation': 22, 'natural': 59, 'blaze': 10, 'burns': 30, 'forgotten': 14, 'revenges': 16, 'bent': 39, 'watch': 150, 'shoot': 30, 'survey': 16, 'captive': 17, 'perfection': 28, 'praising': 10, 'repetition': 10, 'deeper': 14, 'offender': 14, 'hail': 79, 'beams': 26, 'distracted': 14, 'clouds': 61, 'decrees': 10, 'steals': 15, 'stuck': 20, 'herald': 61, 'impression': 12, 'stolen': 35, 'contracted': 10, 'hideous': 21, 'object': 42, 'offend': 61, 'excused': 11, 'strikes': 39, 'turns': 55, 'sour': 38, 'crying': 35, 'unjust': 24, 'destroy': 35, 'waking': 26, 'cries': 70, 'sleeps': 50, 'afternoon': 27, 'knell': 12, 'forget': 96, 'hair': 126, 'hers': 54, 'speaking': 46, 'fasten': 12, 'relieve': 17, 'craft': 21, 'stead': 16, 'pleases': 18, 'reckon': 12, 'thrown': 34, 'contain': 17, 'threw': 31, 'engaged': 18, 'fully': 13, 'satisfaction': 33, 'rough': 81, 'saints': 15, 'falsely': 17, 'makest': 28, 'deadly': 57, 'guards': 10, 'proofs': 14, 'vanity': 21, 'easy': 64, 'dismal': 21, 'vanquish': 14, 'thereto': 18, 'attending': 24, 'visage': 33, 'concern': 12, 'heavens': 157, 'discovery': 15, 'suitors': 18, 'afeard': 31, 'snatch': 16, 'sith': 25, 'wives': 63, 'monsters': 10, 'wretched': 77, 'ancient': 81, 'pitied': 18, 'complaint': 15, 'cease': 50, 'sink': 38, 'behold': 171, 'testament': 12, 'loath': 32, 'attendant': 39, 'spots': 11, 'utter': 47, 'liked': 10, 'cunning': 77, 'subdued': 20, 'market': 41, 'justly': 32, 'diet': 19, 'feather': 31, 'naughty': 15, 'orator': 14, 'knowest': 21, 'motions': 13, 'evidence': 11, 'glove': 39, 'aught': 76, 'prison': 95, 'bail': 14, 'jove': 93, 'accused': 24, 'abuse': 52, 'jeweller': 11, 'abused': 46, 'quit': 49, 'defiled': 10, 'feels': 10, 'truer': 16, 'shadow': 87, 'c': 68, 'appear': 149, 'divorce': 23, 'step': 32, 'flow': 34, 'fresh': 98, 'flower': 63, 'progress': 13, 'epilogue': 13, 'shakespeare': 38, 'homepage': 36, 'entire': 44, 'orchard': 29, 'oliver': 56, 'orlando': 163, 'adam': 40, 'sayest': 38, 'charged': 36, 'school': 40, 'speaks': 136, 'profit': 43, 'stays': 37, 'ox': 13, 'bred': 59, 'manage': 26, 'hired': 10, 'growth': 15, 'countenance': 60, 'lets': 23, 'bars': 13, 'grieves': 23, 'mutiny': 28, 'avoid': 38, 'apart': 41, 'mar': 28, 'idleness': 15, 'naught': 25, 'prodigal': 25, 'spent': 70, 'brothers': 73, 'albeit': 14, 'nearer': 23, 'reverence': 46, 'elder': 40, 'villain': 262, 'youngest': 22, 'rowland': 11, 'begot': 28, 'villains': 48, 'throat': 52, 'saying': 58, 'masters': 109, 'peasant': 23, 'hiding': 11, 'grows': 77, 'troubled': 37, 'holla': 14, 'charles': 118, 'access': 23, 'wrestling': 10, 'banished': 48, 'voluntary': 11, 'exile': 26, 'lands': 56, 'enrich': 17, 'wander': 28, 'rosalind': 274, 'beloved': 57, 'ladies': 143, 'forest': 71, 'robin': 36, 'england': 311, 'flock': 20, 'fleet': 27, 'golden': 101, 'secretly': 12, 'limb': 19, 'acquit': 10, 'foil': 15, 'brook': 85, 'notice': 32, 'herein': 29, 'resolute': 12, 'envious': 36, 'secret': 72, 'discretion': 33, 'lief': 19, 'neck': 81, 'treacherous': 27, 'device': 47, 'ta': 99, 'payment': 16, 'prize': 48, 'hates': 21, 'people': 225, 'clear': 74, 'kindle': 10, 'celia': 125, 'coz': 35, 'mirth': 59, 'teach': 113, 'wouldst': 130, 'rejoice': 26, 'perforce': 48, 'monster': 86, 'devise': 51, 'sports': 11, 'falling': 29, 'earnest': 41, 'wheel': 24, 'bestowed': 13, 'blind': 89, 'mistake': 40, 'goest': 11, 'reigns': 14, 'touchstone': 88, 'flout': 21, 'peradventure': 15, 'wits': 82, 'forsworn': 60, 'heap': 18, 'stroke': 43, 'beards': 14, 'knavery': 18, 'swearing': 23, 'meanest': 20, 'frederick': 31, 'fools': 107, 'wisely': 32, 'beau': 21, 'pigeons': 13, 'princess': 173, 'decree': 12, 'rank': 53, 'amaze': 11, 'manner': 86, 'beginning': 19, 'tale': 124, 'bills': 16, 'necks': 11, 'presents': 14, 'moment': 33, 'ribs': 17, 'third': 259, 'weeping': 72, 'longs': 15, 'music': 214, 'appointed': 34, 'ready': 151, 'entreated': 12, 'crept': 17, 'delight': 67, 'odds': 43, 'challenged': 11, 'others': 211, 'proof': 82, 'adventure': 23, 'punish': 36, 'shamed': 18, 'willing': 39, 'lament': 28, 'injury': 28, 'empty': 57, 'working': 28, 'persuaded': 19, 'invisible': 32, 'shout': 23, 'breathed': 28, 'descended': 16, 'train': 82, 'calling': 17, 'entreaties': 10, 'chain': 69, 'suits': 30, 'block': 17, 'overthrown': 13, 'urged': 31, 'conference': 33, 'weaker': 14, 'applause': 15, 'humorous': 11, 'conceive': 27, 'judge': 96, 'lesser': 31, 'detain': 12, 'sisters': 29, 'niece': 62, 'smother': 21, 'tyrant': 60, 'precious': 78, 'cast': 113, 'curs': 14, 'lame': 16, 'cousins': 16, 'holiday': 14, 'walk': 142, 'coat': 34, 'hem': 10, 'affections': 39, 'despite': 52, 'turning': 28, 'jests': 17, 'sudden': 84, 'ensue': 15, 'hated': 29, 'public': 58, 'miles': 18, 'frantic': 14, 'mistrust': 19, 'whereon': 34, 'likelihood': 14, 'dukedom': 24, 'treason': 102, 'poverty': 22, 'remorse': 30, 'value': 29, 'slept': 36, 'went': 107, 'subtle': 36, 'lips': 164, 'firm': 43, 'doom': 58, 'pronounce': 30, 'grieved': 20, 'cheerful': 17, 'girl': 58, 'maids': 47, 'thieves': 51, 'attire': 14, 'tall': 33, 'points': 28, 'axe': 27, 'thigh': 10, 'boar': 40, 'spear': 13, 'martial': 11, 'outside': 16, 'cowards': 30, 'wide': 55, 'jewels': 31, 'liberty': 71, 'banishment': 40, 'senior': 43, 'amiens': 16, 'mates': 15, 'custom': 48, 'painted': 56, 'woods': 13, 'penalty': 12, 'fang': 14, 'churlish': 17, 'chiding': 11, 'winter': 71, 'bites': 12, 'blows': 78, 'shrink': 12, 'flattery': 23, 'persuade': 35, 'toad': 18, 'ugly': 29, 'exempt': 10, 'haunt': 17, 'tongues': 78, 'trees': 33, 'books': 41, 'stones': 64, 'quiet': 85, 'confines': 10, 'round': 87, 'usurp': 35, 'oak': 28, 'antique': 13, 'root': 49, 'wood': 67, 'hunter': 12, 'heaved': 10, 'discharge': 33, 'piteous': 25, 'extremest': 10, 'swift': 71, 'spectacle': 10, 'needless': 11, 'sum': 53, 'abandon': 10, 'pasture': 12, 'greet': 42, 'sweep': 13, 'fat': 74, 'bankrupt': 15, 'tyrants': 10, 'fright': 36, 'dwelling': 13, 'contemplation': 14, 'cope': 11, 'sullen': 17, 'sufferance': 27, 'wont': 50, 'also': 36, 'graces': 46, 'meeting': 61, 'memory': 54, 'overcome': 11, 'bears': 112, 'doors': 61, 'roof': 18, 'burn': 98, 'lodging': 24, 'abhor': 20, 'food': 66, 'boisterous': 15, 'road': 18, 'subject': 108, 'hire': 18, 'nurse': 217, 'sparrow': 11, 'lusty': 36, 'apply': 15, 'hot': 172, 'frosty': 10, 'necessities': 12, 'appears': 59, 'constant': 53, 'sweat': 42, 'choke': 16, 'tree': 66, 'blossom': 11, 'husbandry': 13, 'wages': 12, 'settled': 18, 'loyalty': 25, 'fourscore': 11, 'debtor': 11, 'jupiter': 36, 'weary': 89, 'doublet': 25, 'ought': 23, 'courageous': 11, 'cross': 99, 'corin': 34, 'silvius': 42, 'partly': 34, 'sigh': 62, 'pillow': 20, 'actions': 31, 'ridiculous': 17, 'drawn': 93, 'fantasy': 15, 'phebe': 46, 'shepherd': 121, 'kissing': 36, 'cow': 10, 'instead': 17, 'lovers': 61, 'stale': 28, 'faint': 52, 'betters': 12, 'faints': 11, 'succor': 14, 'bounds': 23, 'sale': 10, 'absence': 51, 'swain': 24, 'willingly': 35, 'waste': 43, 'soil': 30, 'faithful': 34, 'bird': 51, 'weather': 38, 'suck': 38, 'ragged': 25, 'names': 50, 'compliment': 12, 'beggarly': 10, 'sirs': 44, 'cover': 40, 'matters': 27, 'boast': 36, 'shun': 23, 'seeking': 23, 'gets': 14, 'verse': 18, 'yesterday': 25, 'leaving': 26, 'stubborn': 26, 'greek': 27, 'circle': 14, 'rail': 37, 'egypt': 48, 'banquet': 33, 'savage': 33, 'conceit': 46, 'powers': 87, 'cheerly': 13, 'liest': 46, 'shelter': 16, 'transform': 13, 'beast': 75, 'compact': 16, 'shortly': 47, 'motley': 11, 'miserable': 34, 'drew': 40, 'looking': 44, 'lustre': 14, 'ago': 31, 'eleven': 24, 'ripe': 45, 'rot': 20, 'thereby': 26, 'lungs': 20, 'sans': 16, 'brain': 97, 'dry': 76, 'voyage': 23, 'observation': 11, 'mangled': 12, 'forms': 27, 'provided': 33, 'weed': 18, 'large': 70, 'parish': 10, 'church': 69, 'hit': 62, 'infected': 22, 'patiently': 18, 'fie': 163, 'sting': 31, 'headed': 20, 'licence': 11, 'party': 48, 'sea': 284, 'ebb': 17, 'princes': 116, 'shoulders': 32, 'basest': 11, 'function': 16, 'mettle': 27, 'wild': 98, 'goose': 33, 'forbear': 59, 'cock': 37, 'vein': 18, 'smooth': 49, 'touches': 22, 'fruit': 41, 'gentleness': 17, 'gently': 33, 'stern': 31, 'commandment': 14, 'shade': 26, 'neglect': 24, 'creeping': 11, 'bells': 17, 'eyelids': 12, 'wiped': 14, 'tear': 89, 'bell': 42, 'feasts': 17, 'drops': 58, 'sacred': 49, 'wanting': 22, 'whiles': 84, 'fawn': 14, 'seest': 42, 'universal': 15, 'woeful': 15, 'stage': 34, 'players': 32, 'plays': 36, 'seven': 77, 'ages': 12, 'infant': 27, 'shining': 19, 'snail': 11, 'sighing': 11, 'jealous': 51, 'cannon': 21, 'belly': 43, 'capon': 10, 'formal': 10, 'sixth': 27, 'lean': 52, 'childish': 12, 'treble': 13, 'history': 21, 'taste': 89, 'trouble': 79, 'unkind': 27, 'ingratitude': 21, 'keen': 24, 'heigh': 17, 'green': 117, 'freeze': 10, 'bite': 40, 'nigh': 21, 'cave': 46, 'support': 14, 'candle': 22, 'twelvemonth': 13, 'push': 26, 'officers': 64, 'crowned': 22, 'sway': 42, 'character': 19, 'pleaseth': 17, 'humour': 102, 'plenty': 19, 'philosophy': 14, 'rain': 70, 'sheep': 58, 'complain': 22, 'kindred': 31, 'philosopher': 11, 'damnation': 14, 'whit': 20, 'behavior': 29, 'salute': 20, 'courtiers': 10, 'shepherds': 11, 'briefly': 21, 'ewes': 10, 'mutton': 11, 'wholesome': 33, 'worms': 25, 'raw': 15, 'envy': 60, 'lambs': 13, 'bawd': 91, 'lamb': 40, 'crooked': 15, 'reasonable': 18, 'scape': 39, 'east': 40, 'western': 10, 'mounted': 16, 'fairest': 43, 'eight': 40, 'sleeping': 53, 'slender': 110, 'sweetest': 21, 'verses': 20, 'infect': 22, 'yields': 17, 'writing': 25, 'civil': 51, 'twixt': 75, 'souls': 116, 'teaching': 16, 'cleopatra': 274, 'devised': 21, 'faces': 56, 'prized': 12, 'wearied': 11, 'cried': 74, 'bag': 17, 'feet': 59, 'hanged': 55, 'carved': 10, 'palm': 29, 'irish': 10, 'rat': 11, 'trow': 17, 'mountains': 20, 'wonderful': 24, 'complexion': 40, 'inch': 21, 'south': 28, 'couldst': 32, 'mightst': 29, 'bottle': 26, 'hat': 38, 'chin': 28, 'mocking': 18, 'brow': 76, 'particulars': 10, 'resolve': 33, 'finding': 17, 'relish': 20, 'audience': 33, 'wounded': 41, 'ominous': 10, 'burden': 26, 'tune': 66, 'society': 25, 'wi': 17, 'pleasing': 22, 'answers': 25, 'rings': 19, 'cloth': 19, 'studied': 16, 'nimble': 26, 'worst': 103, 'drowned': 22, 'signior': 132, 'habit': 42, 'forester': 13, 'lazy': 13, 'trot': 11, 'interim': 12, 'latin': 12, 'study': 64, 'gallows': 22, 'softly': 26, 'term': 30, 'moves': 27, 'kindled': 10, 'accent': 16, 'purchase': 35, 'giddy': 31, 'pence': 18, 'recount': 14, 'abuses': 13, 'plants': 14, 'marks': 41, 'rushes': 14, 'prisoner': 82, 'blue': 32, 'neglected': 16, 'revenue': 19, 'sleeve': 22, 'shoe': 16, 'desolation': 11, 'admired': 18, 'rhymes': 11, 'madness': 68, 'madmen': 11, 'ordinary': 12, 'imagine': 26, 'grieve': 41, 'fantastical': 14, 'smiles': 46, 'spit': 30, 'suitor': 23, 'liver': 23, 'spot': 11, 'audrey': 31, 'feature': 14, 'poet': 63, 'goths': 33, 'understanding': 19, 'reckoning': 22, 'truest': 11, 'feign': 10, 'sauce': 15, 'sugar': 21, 'dish': 38, 'couple': 17, 'amen': 82, 'temple': 40, 'assembly': 15, 'beasts': 36, 'necessary': 20, 'goods': 23, 'dowry': 24, 'noblest': 18, 'huge': 37, 'town': 128, 'worthier': 13, 'bachelor': 18, 'chapel': 10, 'toy': 16, 'curb': 15, 'falcon': 13, 'bill': 18, 'wedlock': 11, 'bush': 28, 'excuse': 82, 'wedding': 33, 'dissembling': 14, 'judas': 18, 'bread': 27, 'pair': 46, 'certainly': 26, 'worm': 44, 'eaten': 24, 'downright': 13, 'tapster': 14, 'asked': 15, 'parentage': 17, 'laughed': 14, 'breaks': 35, 'staff': 44, 'guides': 10, 'pageant': 10, 'glow': 14, 'remove': 35, 'busy': 30, 'executioner': 14, 'murder': 160, 'frail': 21, 'gates': 101, 'murderers': 22, 'frown': 53, 'swoon': 12, 'wounds': 93, 'arrows': 13, 'afflict': 15, 'mocks': 20, 'silk': 21, 'knees': 54, 'fasting': 14, 'fast': 144, 'frowning': 15, 'relief': 13, 'employ': 49, 'plenteous': 12, 'harvest': 22, 'talks': 14, 'faster': 17, 'heal': 15, 'passing': 36, 'laughing': 17, 'extremity': 30, 'abominable': 15, 'censure': 30, 'scholar': 21, 'emulation': 11, 'musician': 31, 'lawyer': 10, 'objects': 11, 'blank': 13, 'nativity': 14, 'divide': 27, 'shoulder': 32, 'tardy': 14, 'beholding': 32, 'armed': 46, 'slander': 54, 'maker': 16, 'occasion': 77, 'entreaty': 12, 'talking': 26, 'person': 148, 'troilus': 250, 'grecian': 31, 'hero': 122, 'turned': 36, 'winged': 17, 'possessed': 12, 'april': 15, 'changes': 19, 'clamorous': 11, 'ape': 23, 'fountain': 17, 'disposed': 26, 'inclined': 18, 'chimney': 11, 'flattering': 22, 'jot': 22, 'hollow': 56, 'chosen': 22, 'offenders': 12, 'prate': 15, 'sounded': 31, 'bastard': 192, 'venus': 30, 'spleen': 29, 'roman': 119, 'conqueror': 23, 'victory': 50, 'leather': 13, 'crest': 29, 'bore': 33, 'errand': 18, 'angry': 104, 'tenor': 19, 'guiltless': 24, 'hare': 22, 'hunt': 38, 'verily': 13, 'gloves': 17, 'giant': 18, 'cruelty': 20, 'vengeance': 54, 'eyne': 13, 'raise': 50, 'alack': 77, 'mild': 38, 'aspect': 27, 'chid': 20, 'endured': 20, 'snake': 12, 'west': 34, 'female': 17, 'sends': 41, 'napkin': 14, 'lo': 83, 'bald': 18, 'gilded': 18, 'threats': 16, 'seeing': 58, 'unnatural': 39, 'hungry': 26, 'purposed': 21, 'kindness': 57, 'rescued': 11, 'array': 16, 'instantly': 24, 'torn': 15, 'small': 98, 'claim': 66, 'interest': 33, 'william': 80, 'cup': 50, 'filling': 10, 'vulgar': 21, 'steel': 75, 'faction': 19, 'tremble': 39, 'depart': 69, 'trip': 13, 'enjoy': 57, 'invite': 21, 'followers': 51, 'prepare': 67, 'handkerchief': 36, 'wonders': 17, 'brag': 13, 'overcame': 10, 'sought': 45, 'degrees': 18, 'stairs': 11, 'climb': 25, 'clubs': 10, 'nuptial': 20, 'heaviness': 18, 'belief': 14, 'profound': 14, 'tomorrow': 13, 'human': 27, 'sober': 21, 'ungentle': 12, 'impatience': 19, 'purity': 11, 'howling': 12, 'wolves': 18, 'moon': 140, 'satisfied': 50, 'joyful': 31, 'dishonest': 11, 'pages': 11, 'middle': 20, 'roundly': 10, 'hey': 27, 'corn': 38, 'spring': 74, 'birds': 43, 'folks': 13, 'voices': 54, 'kingdoms': 25, 'refuse': 31, 'bargain': 21, 'doubts': 13, 'lively': 13, 'tutor': 22, 'studies': 13, 'obscured': 12, 'flood': 56, 'minded': 17, 'trod': 14, 'quarrels': 18, 'seventh': 10, 'press': 48, 'according': 47, 'dwells': 19, 'pearl': 25, 'bolt': 13, 'diseases': 23, 'dislike': 18, 'reply': 24, 'reproof': 20, 'measured': 12, 'swords': 71, 'print': 15, 'book': 98, 'fifth': 32, 'circumstance': 37, 'parties': 15, 'shook': 30, 'hymen': 13, 'bar': 32, 'confusion': 35, 'conclusion': 30, 'events': 15, 'bands': 15, 'finish': 14, 'board': 20, 'degree': 47, 'fairly': 46, 'potent': 20, 'begun': 33, 'number': 56, 'returned': 18, 'states': 12, 'meantime': 25, 'n': 78, 'measures': 12, 'former': 58, 'bequeath': 16, 'pleasures': 37, 'dancing': 20, 'rites': 22, 'prologue': 38, 'conjure': 28, 'comedy': 12, 'errors': 11, 'hall': 38, 'solinus': 26, 'aegeon': 24, 'gaoler': 40, 'procure': 19, 'woes': 57, 'merchant': 62, 'syracuse': 228, 'laws': 39, 'enmity': 21, 'sprung': 13, 'outrage': 14, 'merchants': 10, 'dealing': 18, 'countrymen': 40, 'redeem': 28, 'statutes': 11, 'threatening': 21, 'decreed': 10, 'admit': 23, 'traffic': 12, 'towns': 29, 'ephesus': 173, 'dispose': 24, 'levied': 14, 'substance': 38, 'valued': 14, 'condemned': 13, 'likewise': 27, 'heavier': 14, 'task': 38, 'wrought': 43, 'hap': 26, 'punishment': 21, 'safe': 87, 'distinguish': 11, 'inn': 21, 'meaner': 15, 'male': 13, 'alike': 43, 'daily': 31, 'unwilling': 11, 'agreed': 18, 'aboard': 35, 'league': 36, 'sail': 50, 'retain': 10, 'minds': 60, 'doubtful': 23, 'embraced': 14, 'mourn': 31, 'forced': 46, 'delays': 10, 'sailors': 18, 'boat': 11, 'ship': 63, 'careful': 25, 'mast': 11, 'storms': 19, 'gazing': 21, 'dispersed': 14, 'vapours': 11, 'benefit': 37, 'calm': 40, 'ships': 29, 'amain': 15, 'sequel': 10, 'worthily': 14, 'leagues': 12, 'borne': 84, 'midst': 13, 'woe': 159, 'seized': 18, 'guests': 25, 'bark': 50, 'sever': 11, 'bliss': 21, 'befall': 28, 'greece': 22, 'hapless': 10, 'fates': 16, 'dire': 21, 'sue': 27, 'mart': 17, 'antipholus': 215, 'dromio': 219, 'gaze': 27, 'stiff': 17, 'dine': 25, 'crave': 48, 'consort': 12, 'commends': 21, 'ocean': 30, 'confounds': 12, 'chance': 106, 'pig': 11, 'twelve': 49, 'penitent': 14, 'sixpence': 12, 'wednesday': 15, 'dally': 12, 'prays': 20, 'killing': 28, 'witches': 13, 'prating': 12, 'adriana': 88, 'luciana': 55, 'perhaps': 47, 'fret': 15, 'headstrong': 10, 'fishes': 13, 'subjects': 67, 'watery': 23, 'troubles': 16, 'pause': 43, 'bruised': 10, 'mate': 16, 'urging': 10, 'bereft': 21, 'begg': 26, 'beshrew': 31, 'struck': 83, 'plainly': 19, 'stark': 10, 'beating': 30, 'spurn': 29, 'starve': 19, 'wasted': 18, 'barren': 39, 'discourse': 64, 'marble': 19, 'bait': 16, 'ruin': 50, 'repair': 45, 'unruly': 15, 'feeds': 21, 'jealousy': 38, 'dispense': 10, 'homage': 16, 'touching': 27, 'falsehood': 34, 'corruption': 17, 'alter': 34, 'strokes': 23, 'madly': 17, 'chat': 15, 'shines': 31, 'creep': 28, 'hides': 17, 'amends': 24, 'choleric': 10, 'conclude': 30, 'incorporate': 10, 'gulf': 10, 'ruffian': 17, 'hurl': 11, 'possess': 67, 'blot': 26, 'crime': 13, 'digest': 14, 'truce': 15, 'dame': 36, 'converse': 21, 'drift': 19, 'gravity': 13, 'sap': 14, 'theme': 31, 'spread': 31, 'beads': 11, 'fairy': 45, 'spite': 62, 'form': 105, 'porter': 42, 'advised': 40, 'angelo': 211, 'linger': 10, 'shop': 12, 'drunkard': 13, 'ink': 30, 'cheap': 15, 'dainty': 21, 'affords': 15, 'guest': 40, 'hatch': 19, 'wenches': 22, 'dined': 17, 'luce': 14, 'proverb': 14, 'minion': 17, 'ache': 18, 'sore': 52, 'thin': 26, 'warm': 54, 'buck': 19, 'ope': 32, 'fowl': 15, 'iron': 47, 'compass': 36, 'suspect': 40, 'ruled': 37, 'tiger': 19, 'stirring': 23, 'comment': 12, 'rout': 10, 'expense': 17, 'springs': 16, 'building': 11, 'vice': 55, 'carriage': 23, 'truant': 13, 'feeble': 38, 'silver': 85, 'waves': 20, 'hairs': 33, 'glorious': 47, 'wink': 33, 'beastly': 20, 'rags': 16, 'grain': 18, 'nell': 16, 'globe': 11, 'ireland': 31, 'scotland': 28, 'rheum': 14, 'ran': 34, 'spain': 12, 'oh': 33, 'privy': 15, 'mole': 10, 'wart': 17, 'witch': 91, 'shore': 71, 'harbour': 22, 'pack': 31, 'supper': 64, 'visit': 57, 'streets': 53, 'meets': 16, 'attach': 15, 'growing': 31, 'courtezan': 19, 'goldsmith': 10, 'rope': 17, 'confederates': 10, 'pound': 55, 'holp': 19, 'odd': 34, 'ducats': 52, 'thereof': 36, 'tide': 54, 'bringing': 22, 'shrew': 16, 'brawl': 15, 'arrest': 27, 'trim': 22, 'madman': 23, 'drunken': 23, 'arrested': 12, 'tempt': 31, 'persuasion': 20, 'deformed': 14, 'faced': 53, 'blunt': 69, 'wail': 27, 'curse': 106, 'garment': 23, 'fiend': 65, 'fury': 70, 'wolf': 38, 'hound': 18, 'draws': 40, 'redemption': 10, 'debt': 46, 'sergeant': 19, 'fondly': 10, 'stealing': 17, 'immediately': 21, 'picture': 46, 'calf': 32, 'tired': 24, 'rests': 15, 'sets': 28, 'expedition': 25, 'distract': 10, 'dam': 29, 'expect': 38, 'diamond': 15, 'devils': 41, 'cherry': 10, 'avaunt': 17, 'forty': 30, 'denies': 13, 'rage': 133, 'entrance': 22, 'lunatic': 12, 'wayward': 14, 'lightly': 12, 'whoreson': 40, 'sensible': 25, 'waked': 21, 'prophecy': 15, 'ecstasy': 15, 'striking': 14, 'doting': 16, 'distressed': 10, 'revel': 16, 'slanders': 16, 'vigour': 12, 'yielding': 20, 'humours': 32, 'frenzy': 13, 'loathsome': 20, 'abject': 11, 'shameful': 18, 'forthwith': 24, 'entered': 11, 'idly': 16, 'rapier': 29, 'naked': 54, 'stuff': 62, 'mountain': 43, 'reverend': 62, 'walks': 26, 'scandal': 21, 'imprisonment': 16, 'staying': 13, 'controversy': 11, 'wretch': 58, 'livest': 16, 'resort': 14, 'impeach': 11, 'defy': 28, 'aemelia': 16, 'throng': 24, 'different': 11, 'brake': 12, 'stray': 15, 'venom': 26, 'clamours': 11, 'raging': 20, 'fever': 22, 'moody': 15, 'grim': 26, 'disturb': 20, 'wildly': 13, 'enters': 19, 'sanctuary': 13, 'holiness': 13, 'rise': 87, 'execution': 48, 'abbey': 20, 'publicly': 10, 'bondman': 10, 'houses': 30, 'bearing': 84, 'committed': 37, 'wot': 28, 'chased': 12, 'pursued': 16, 'determine': 24, 'mire': 10, 'scars': 16, 'gavest': 16, 'beyond': 62, 'grievous': 25, 'tells': 47, 'perjured': 27, 'provoked': 17, 'rabble': 13, 'eyed': 24, 'pernicious': 21, 'feeling': 38, 'vault': 17, 'bonds': 22, 'freedom': 33, 'shames': 17, 'miracle': 28, 'coldly': 15, 'vouchsafe': 52, 'cords': 12, 'whatsoever': 17, 'crack': 49, 'hid': 49, 'deaf': 26, 'witnesses': 12, 'patron': 10, 'during': 19, 'dangers': 21, 'husbands': 27, 'ghost': 81, 'fatal': 48, 'semblance': 25, 'warrior': 13, 'renowned': 34, 'pawn': 36, 'assembled': 10, 'travail': 10, 'gossip': 12, 'cuts': 14, 'cymbeline': 117, 'britain': 36, 'garden': 48, 'frowns': 20, 'kingdom': 105, 'imprison': 16, 'compare': 24, 'crush': 14, 'sicilius': 10, 'romans': 56, 'titles': 23, 'glory': 96, 'leonatus': 118, 'deceased': 14, 'babe': 42, 'protection': 13, 'posthumus': 126, 'stol': 26, 'negligence': 15, 'imogen': 162, 'keys': 15, 'pangs': 15, 'tickle': 11, 'abide': 38, 'loyal': 34, 'plight': 17, 'rome': 342, 'philario': 20, 'gall': 37, 'injuries': 21, 'riding': 10, 'petty': 32, 'exchange': 32, 'fraught': 11, 'protect': 14, 'disloyal': 12, 'obedience': 52, 'chose': 18, 'eagle': 30, 'seat': 60, 'baseness': 17, 'added': 18, 'restore': 19, 'neat': 18, 'pisanio': 104, 'needle': 11, 'haven': 18, 'cloten': 110, 'shirt': 15, 'violence': 19, 'sacrifice': 23, 'abroad': 62, 'inches': 11, 'sign': 55, 'grew': 33, 'shores': 21, 'dst': 23, 'linen': 20, 'happier': 14, 'deck': 24, 'stirs': 17, 'strings': 17, 'wept': 33, 'vantage': 40, 'morn': 42, 'noon': 20, 'tyrannous': 12, 'north': 45, 'buds': 13, 'iachimo': 95, 'expected': 11, 'marrying': 11, 'approbation': 14, 'lamentable': 19, 'battery': 16, 'flat': 25, 'orleans': 86, 'mended': 13, 'confounded': 10, 'prefer': 20, 'abate': 15, 'beheld': 35, 'rated': 17, 'brace': 17, 'contains': 11, 'notwithstanding': 22, 'admittance': 10, 'opportunity': 21, 'wager': 23, 'assail': 11, 'million': 14, 'speeches': 22, 'undergo': 18, 'exceeds': 11, 'sufficient': 19, 'articles': 19, 'directly': 31, 'cornelius': 21, 'dew': 46, 'flowers': 68, 'conscience': 123, 'poisonous': 13, 'pupil': 11, 'proceeded': 13, 'devilish': 14, 'forces': 42, 'hanging': 43, 'lingering': 13, 'speechless': 13, 'decay': 23, 'built': 19, 'takest': 11, 'preferment': 11, 'lt': 23, 'load': 24, 'sly': 41, 'agent': 10, 'closet': 29, 'wills': 29, 'flying': 18, 'tied': 39, 'aloud': 29, 'arch': 16, 'appetite': 37, 'excellence': 19, 'opposed': 28, 'pleasant': 36, 'incline': 10, 'account': 45, 'talents': 16, 'discern': 10, 'spur': 33, 'mount': 36, 'capitol': 45, 'stinking': 11, 'beggary': 12, 'charms': 22, 'partner': 18, 'coffers': 13, 'revenged': 46, 'stock': 32, 'sheets': 20, 'solicit': 13, 'deeply': 18, 'sits': 64, 'mongst': 22, 'unlike': 15, 'emperor': 103, 'plate': 11, 'exquisite': 12, 'trunk': 25, 'gallia': 12, 'bowl': 15, 'fighting': 22, 'nobody': 21, 'issues': 10, 'crafty': 12, 'govern': 44, 'plots': 15, 'hateful': 46, 'horrid': 16, 'fold': 32, 'leaf': 17, 'taper': 14, 'burning': 61, 'awake': 81, 'fairies': 37, 'tarquin': 44, 'lily': 19, 'bows': 14, 'lights': 33, 'windows': 18, 'arras': 13, 'figures': 15, 'slippery': 11, 'spotted': 16, 'crimson': 15, 'raven': 22, 'ante': 10, 'temper': 47, 'furious': 15, 'musicians': 21, 'conceited': 10, 'phoebus': 18, 'arise': 39, 'steeds': 12, 'winking': 11, 'guts': 15, 'eunuch': 13, 'amend': 18, 'services': 33, 'duties': 22, 'ambassadors': 16, 'caius': 148, 'lucius': 241, 'knocks': 25, 'buys': 11, 'telling': 29, 'regard': 47, 'silent': 36, 'charity': 60, 'alms': 15, 'dishes': 13, 'knit': 39, 'squire': 16, 'profane': 18, 'groom': 28, 'hangman': 21, 'mischance': 23, 'named': 24, 'frighted': 16, 'europe': 10, 'discontent': 17, 'quake': 16, 'tribute': 26, 'legions': 19, 'landed': 15, 'julius': 18, 'smiled': 12, 'discipline': 14, 'winds': 54, 'sails': 19, 'journey': 28, 'apparent': 27, 'circumstances': 11, 'confirm': 39, 'watching': 10, 'banks': 17, 'rarely': 11, 'likely': 28, 'kills': 31, 'corrupted': 21, 'whore': 48, 'fiends': 16, 'meal': 12, 'government': 31, 'stamp': 32, 'restrain': 12, 'yellow': 26, 'oppose': 18, 'changing': 12, 'detest': 10, 'feats': 14, 'pounds': 16, 'noses': 14, 'kings': 129, 'isle': 40, 'neptune': 23, 'park': 30, 'rocks': 20, 'roaring': 26, 'sands': 45, 'conquest': 32, 'coast': 25, 'fires': 24, 'britons': 11, 'moe': 24, 'gripe': 11, 'injurious': 15, 'yoke': 37, 'ordain': 10, 'utterance': 11, 'precedent': 21, 'afterwards': 17, 'girdle': 11, 'crows': 19, 'infection': 19, 'tongued': 11, 'prevail': 42, 'humanity': 12, 'characters': 13, 'asunder': 15, 'bees': 11, 'locks': 22, 'tables': 13, 'renew': 15, 'milford': 19, 'wales': 42, 'gap': 11, 'rode': 17, 'belarius': 80, 'guiderius': 78, 'arviragus': 65, 'stoop': 40, 'hill': 44, 'tales': 23, 'courts': 10, 'apprehend': 17, 'sweeter': 15, 'cell': 41, 'toil': 32, 'slanderous': 12, 'record': 21, 'storm': 62, 'pious': 11, 'dreams': 45, 'princely': 82, 'game': 24, 'stare': 11, 'inward': 32, 'wildness': 10, 'bleeding': 37, 'pandar': 22, 'painting': 18, 'aeneas': 84, 'sinon': 10, 'wretchedness': 10, 'mansion': 12, 'riches': 22, 'slaughter': 64, 'afore': 15, 'strain': 38, 'entreats': 15, 'butcher': 30, 'knife': 63, 'wake': 56, 'cursed': 54, 'bide': 15, 'ado': 22, 'volume': 11, 'swan': 12, 'ambassador': 25, 'its': 24, 'harder': 12, 'cloak': 30, 'serving': 33, 'manhood': 26, 'wrote': 10, 'ourself': 26, 'omit': 14, 'horsemen': 11, 'readiness': 15, 'noted': 23, 'retired': 13, 'chambers': 13, 'whereto': 24, 'proffer': 11, 'british': 12, 'rages': 13, 'throwing': 19, 'choked': 13, 'perish': 34, 'hum': 21, 'follower': 10, 'belch': 176, 'ravish': 21, 'torment': 29, 'vex': 29, 'execute': 22, 'despised': 25, 'duteous': 10, 'resolution': 33, 'helps': 17, 'relieved': 12, 'beggars': 26, 'path': 15, 'famine': 11, 'scarcely': 16, 'foe': 72, 'cook': 18, 'stomachs': 15, 'flint': 27, 'paragon': 11, 'dirt': 11, 'fidele': 10, 'bigger': 15, 'laying': 16, 'peer': 19, 'twain': 42, 'dress': 30, 'owl': 26, 'senators': 61, 'tribunes': 58, 'senator': 109, 'levy': 11, 'absolute': 33, 'tribune': 17, 'numbers': 40, 'confer': 15, 'lines': 28, 'enforced': 30, 'usage': 11, 'hunting': 20, 'clay': 16, 'citizen': 187, 'stick': 30, 'rob': 39, 'quantity': 13, 'imperious': 14, 'rivers': 62, 'roots': 14, 'smiling': 33, 'mingle': 14, 'companies': 11, 'answering': 10, 'grandfather': 18, 'varlet': 22, 'adder': 15, 'spider': 11, 'slain': 134, 'apprehension': 15, 'threat': 12, 'suffering': 16, 'tail': 26, 'perilous': 11, 'wave': 14, 'hasty': 22, 'pine': 20, 'instinct': 15, 'royalty': 25, 'sow': 16, 'embassy': 13, 'sounds': 46, 'accidents': 13, 'triumphs': 12, 'lamenting': 10, 'toys': 15, 'crutch': 11, 'steps': 30, 'veins': 30, 'heirs': 25, 'corse': 29, 'priests': 16, 'tween': 16, 'thersites': 122, 'ajax': 130, 'worldly': 15, 'lads': 23, 'lightning': 26, 'thunder': 78, 'moan': 18, 'witchcraft': 18, 'charm': 41, 'bout': 10, 'graves': 33, 'wither': 47, 'bedfellow': 10, 'keeper': 52, 'imagined': 10, 'curses': 30, 'hecuba': 15, 'greeks': 30, 'pregnant': 16, 'murderous': 22, 'soothsayer': 41, 'senate': 43, 'stirr': 14, 'captains': 23, 'vision': 17, 'vanish': 22, 'sins': 34, 'demanded': 10, 'richard': 539, 'approve': 25, 'consul': 32, 'dig': 10, 'weeds': 43, 'wipe': 26, 'diligence': 11, 'depend': 14, 'supply': 28, 'withdraw': 36, 'annoy': 12, 'chances': 14, 'barbarous': 21, 'secure': 34, 'aye': 16, 'cradle': 17, 'slaves': 39, 'rider': 10, 'permit': 13, 'dread': 54, 'thrift': 12, 'habits': 14, 'guilt': 34, 'lane': 13, 'disorder': 15, 'strangely': 26, 'betimes': 24, 'backs': 22, 'hearted': 39, 'ditch': 10, 'wall': 79, 'nobleness': 18, 'lance': 16, 'lions': 24, 'dying': 52, 'resist': 13, 'grown': 60, 'mockery': 14, 'preserved': 15, 'beds': 28, 'knives': 10, 'silly': 24, 'peck': 12, 'coin': 22, 'apparition': 12, 'leading': 20, 'womb': 58, 'fruitful': 16, 'taint': 15, 'maintain': 48, 'crystal': 16, 'exercise': 21, 'race': 15, 'ghosts': 15, 'appeal': 19, 'throws': 26, 'region': 17, 'planted': 11, 'shadows': 29, 'reign': 53, 'affliction': 21, 'confine': 16, 'din': 10, 'celestial': 18, 'grandsire': 28, 'wretches': 14, 'steep': 21, 'stately': 11, 'cedar': 13, 'branches': 17, 'revive': 10, 'sympathy': 12, 'tavern': 15, 'thousands': 17, 'directed': 12, 'jump': 13, 'beget': 24, 'knights': 50, 'companions': 17, 'dignities': 14, 'horror': 18, 'affected': 21, 'abhorr': 11, 'fitted': 13, 'beautiful': 15, 'prisoners': 42, 'comest': 30, 'accident': 22, 'boon': 11, 'lad': 28, 'scorns': 22, 'joys': 32, 'sand': 11, 'torments': 12, 'conceal': 17, 'accursed': 33, 'hook': 14, 'whereat': 18, 'car': 14, 'vilely': 16, 'wounding': 12, 'whereupon': 14, 'murderer': 135, 'upright': 23, 'caused': 12, 'fares': 34, 'compound': 18, 'offices': 30, 'posts': 12, 'provoke': 18, 'descent': 17, 'loins': 12, 'arts': 13, 'mantle': 19, 'swallow': 31, 'fierce': 39, 'harmless': 20, 'forlorn': 21, 'graced': 17, 'beseeming': 12, 'apt': 43, 'oracle': 27, 'lofty': 17, 'wonted': 12, 'fingers': 41, 'harmony': 23, 'aloft': 20, 'navarre': 11, 'ferdinand': 176, 'biron': 193, 'longaville': 59, 'brazen': 13, 'smallest': 17, 'strict': 22, 'expressly': 11, 'stops': 21, 'purchased': 15, 'continual': 14, 'geese': 11, 'frost': 12, 'infants': 10, 'penance': 18, 'item': 31, 'proclaimed': 11, 'article': 19, 'complete': 19, 'armado': 122, 'relate': 10, 'tawny': 13, 'costard': 118, 'concerning': 12, 'jaquenetta': 29, 'correction': 20, 'defend': 71, 'simplicity': 13, 'deputy': 24, 'welkin': 19, 'coloured': 10, 'walked': 10, 'continent': 12, 'wherewith': 10, 'pricks': 12, 'repute': 12, 'don': 328, 'adriano': 111, 'prosperity': 16, 'moth': 103, 'crosses': 13, 'drawing': 25, 'carrying': 10, 'assist': 22, 'blushing': 24, 'newly': 31, 'heavily': 11, 'pent': 11, 'tempted': 15, 'butt': 16, 'shaft': 13, 'subdue': 10, 'rosaline': 101, 'maria': 117, 'katharine': 146, 'boyet': 101, 'summon': 17, 'parley': 31, 'perfections': 11, 'painful': 10, 'seemeth': 11, 'moving': 21, 'personal': 14, 'signify': 25, 'beauteous': 34, 'alencon': 48, 'ornaments': 13, 'tire': 17, 'mask': 25, 'requests': 10, 'liberal': 29, 'choler': 27, 'offering': 10, 'agree': 20, 'thorough': 14, 'impatient': 20, 'pavilion': 11, 'singing': 40, 'message': 33, 'l': 27, 'envoy': 16, 'salve': 11, 'provokes': 10, 'obscure': 12, 'ending': 16, 'impose': 10, 'remuneration': 12, 'jew': 75, 'beadle': 15, 'pedant': 36, 'regent': 20, 'anointed': 19, 'pitch': 35, 'joan': 83, 'paint': 17, 'curst': 27, 'member': 10, 'den': 27, 'waist': 15, 'bearer': 11, 'mistook': 22, 'lovely': 48, 'robes': 21, 'finely': 10, 'miscarry': 13, 'challenge': 50, 'sola': 10, 'holofernes': 62, 'nathaniel': 36, 'crab': 14, 'inclination': 14, 'goodman': 14, 'capacity': 11, 'argues': 11, 'shapes': 32, 'daughters': 56, 'schoolmaster': 10, 'venice': 82, 'non': 15, 'la': 123, 'bias': 12, 'framed': 23, 'miscarried': 10, 'saith': 12, 'text': 15, 'glasses': 11, 'listen': 15, 'likeness': 24, 'papers': 15, 'fellowship': 19, 'empress': 50, 'perjury': 21, 'shine': 46, 'kate': 108, 'playing': 22, 'triumph': 51, 'zeal': 35, 'leap': 37, 'tush': 24, 'nestor': 72, 'timon': 339, 'mess': 11, 'purses': 12, 'dismiss': 30, 'gorgeous': 10, 'peremptory': 13, 'cull': 11, 'worthies': 15, 'passes': 22, 'infancy': 12, 'badge': 15, 'hue': 25, 'imitate': 12, 'upward': 13, 'author': 17, 'elements': 21, 'courses': 18, 'stopp': 25, 'proves': 17, 'apollo': 27, 'lute': 18, 'drowsy': 17, 'nourish': 12, 'advance': 32, 'conflict': 10, 'revels': 18, 'dances': 10, 'picked': 11, 'thread': 20, 'anne': 175, 'et': 32, 'basket': 37, 'dragon': 12, 'infamy': 20, 'dunghill': 10, 'multitude': 19, 'generous': 10, 'impart': 12, 'secrecy': 16, 'assistance': 13, 'joint': 30, 'pompey': 205, 'thumb': 14, 'hiss': 12, 'tabour': 10, 'madame': 10, 'sheet': 15, 'grandam': 31, 'mouse': 19, 'moreover': 20, 'pearls': 15, 'bootless': 20, 'fate': 50, 'stabb': 15, 'arguments': 12, 'clapp': 11, 'wag': 19, 'elbow': 58, 'toe': 16, 'parle': 13, 'gallants': 13, 'theirs': 31, 'merriment': 13, 'counsels': 18, 'intended': 18, 'trumpet': 89, 'beauties': 12, 'dames': 11, 'visitation': 17, 'moonshine': 21, 'visor': 12, 'dice': 10, 'swifter': 13, 'frozen': 13, 'tapers': 11, 'puff': 12, 'tonight': 19, 'cases': 12, 'caps': 21, 'bud': 17, 'gear': 11, 'presented': 16, 'wakes': 18, 'pins': 10, 'chides': 11, 'bone': 21, 'construe': 11, 'rebuke': 19, 'brass': 16, 'silken': 18, 'flaw': 10, 'visited': 10, 'intend': 56, 'despise': 19, 'dash': 16, 'trencher': 11, 'dick': 33, 'signs': 37, 'terror': 34, 'apple': 13, 'shroud': 12, 'fray': 18, 'actors': 12, 'hector': 223, 'alexander': 31, 'smells': 12, 'dismay': 17, 'retires': 22, 'sort': 71, 'george': 58, 'drawer': 18, 'achilles': 170, 'painter': 48, 'gilt': 16, 'rein': 18, 'yard': 14, 'sup': 20, 'pole': 22, 'northern': 11, 'incensed': 22, 'combat': 27, 'cloud': 31, 'boldly': 17, 'causes': 15, 'mourning': 25, 'pierce': 27, 'roll': 12, 'glance': 11, 'latest': 15, 'deserts': 24, 'flatter': 57, 'hearers': 11, 'plough': 12, 'pipe': 17, 'nightly': 22, 'pot': 17, 'mercury': 15, 'vincentio': 253, 'lists': 18, 'vienna': 10, 'torches': 18, 'lends': 14, 'importune': 12, 'hopeful': 11, 'lucio': 135, 'dukes': 21, 'pirate': 12, 'proportion': 30, 'overdone': 24, 'claudio': 286, 'fooling': 10, 'river': 25, 'seed': 14, 'trade': 43, 'thomas': 73, 'provost': 116, 'juliet': 199, 'rats': 10, 'lechery': 18, 'mutual': 15, 'governor': 26, 'fills': 13, 'armour': 48, 'wrinkled': 14, 'biting': 11, 'rod': 15, 'plucks': 15, 'baby': 17, 'beats': 31, 'isabella': 161, 'farther': 27, 'prayer': 39, 'affair': 12, 'pattern': 16, 'executed': 19, 'confessor': 10, 'froth': 26, 'commonweal': 10, 'void': 11, 'respected': 14, 'iniquity': 12, 'jade': 14, 'relent': 16, 'marshal': 32, 'robe': 19, 'isabel': 31, 'bethink': 21, 'dared': 10, 'split': 20, 'carrion': 13, 'swelling': 25, 'conception': 10, 'awe': 22, 'vices': 14, 'louder': 11, 'display': 14, 'accusation': 13, 'mouths': 39, 've': 15, 'servile': 13, 'bowels': 22, 'deaths': 21, 'intends': 17, 'appointment': 10, 'cleave': 11, 'determined': 19, 'follies': 13, 'filth': 10, 'frankly': 10, 'horrible': 25, 'loathed': 19, 'defiance': 12, 'bending': 18, 'require': 21, 'rid': 34, 'frailty': 12, 'substitute': 11, 'wronged': 25, 'solemnity': 11, 'continuance': 10, 'impediment': 15, 'current': 27, 'brown': 21, 'worser': 19, 'warning': 14, 'spy': 29, 'wheels': 10, 'images': 11, 'beef': 13, 'eating': 14, 'drinking': 29, 'ruthless': 11, 'opposite': 29, 'philip': 66, 'security': 16, 'temperance': 11, 'seals': 10, 'wary': 10, 'rack': 20, 'fancies': 14, 'barnardine': 25, 'abhorson': 20, 'occupation': 10, 'members': 10, 'yare': 10, 'knocking': 38, 'avouch': 11, 'receives': 13, 'amazement': 13, 'throats': 27, 'drinks': 24, 'generation': 12, 'peter': 90, 'bawdy': 14, 'redress': 31, 'injustice': 10, 'proceedings': 19, 'bulk': 16, 'afoot': 22, 'flavius': 65, 'veil': 13, 'wring': 13, 'blemish': 14, 'soundly': 23, 'yesternight': 12, 'depose': 10, 'instruments': 27, 'handle': 12, 'mum': 12, 'unhallow': 11, 'strangeness': 12, 'reproach': 26, 'bounteous': 17, 'lewd': 14, 'windsor': 32, 'hugh': 131, 'evans': 106, 'robert': 39, 'gloucester': 723, 'r': 22, 'falstaff': 596, 'riot': 14, 'abraham': 10, 'cur': 32, 'bardolph': 207, 'nym': 57, 'pistol': 185, 'tut': 40, 'catching': 12, 'rascals': 18, 'edward': 403, 'scarlet': 13, 'ford': 287, 'shillings': 10, 'alice': 29, 'oman': 11, 'fence': 15, 'unmannerly': 13, 'bully': 20, 'lime': 15, 'jerkin': 12, 'anchor': 10, 'rises': 12, 'pandarus': 188, 'dove': 28, 'incense': 11, 'rugby': 32, 'coal': 21, 'vat': 12, 'il': 15, 'je': 29, 'dat': 25, 'vill': 10, 'gar': 31, 'weapon': 29, 'fenton': 45, 'thinkest': 15, 'nan': 11, 'scaped': 14, 'sack': 61, 'parliament': 25, 'hack': 13, 'daylight': 11, 'tempest': 53, 'ashore': 13, 'opinions': 14, 'appoint': 12, 'prevent': 29, 'commended': 12, 'paltry': 11, 'liquor': 18, 'welsh': 26, 'weapons': 42, 'burnt': 21, 'gratis': 10, 'proudest': 13, 'earls': 10, 'fights': 15, 'cudgel': 16, 'francisco': 16, 'bodies': 46, 'englishman': 11, 'farm': 10, 'feasting': 10, 'prosper': 24, 'cowardly': 17, 'leader': 14, 'shower': 11, 'revolted': 12, 'wilful': 16, 'neighbours': 24, 'cue': 12, 'assurance': 29, 'poins': 95, 'waits': 12, 'muddy': 12, 'direction': 24, 'unwholesome': 10, 'period': 17, 'suspicion': 34, 'ascend': 13, 'jealousies': 11, 'castle': 99, 'breakfast': 16, 'bags': 12, 'gloucestershire': 12, 'slack': 13, 'swells': 10, 'sped': 10, 'distraction': 11, 'stockings': 13, 'suffered': 14, 'thaw': 10, 'declined': 10, 'hag': 11, 'mankind': 19, 'utterly': 10, 'chest': 10, 'mischief': 45, 'aunt': 26, 'conspiracy': 11, 'element': 14, 'mass': 32, 'submission': 11, 'herne': 10, 'blasts': 10, 'waxen': 11, 'practised': 15, 'descend': 25, 'beguiled': 21, 'cozened': 10, 'maidenhead': 13, 'melt': 33, 'quaint': 11, 'spies': 15, 'divinity': 10, 'pit': 25, 'treachery': 24, 'bull': 22, 'grey': 81, 'airy': 13, 'juice': 10, 'balm': 19, 'fertile': 16, 'purple': 19, 'knighthood': 14, 'disperse': 10, 'walking': 17, 'realm': 53, 'entrails': 11, 'arrow': 10, 'salarino': 37, 'salanio': 25, 'ports': 13, 'misfortune': 13, 'ague': 12, 'wealthy': 11, 'andrew': 117, 'burial': 22, 'trusted': 11, 'bassanio': 122, 'lorenzo': 80, 'gratiano': 100, 'commendable': 10, 'innocence': 23, 'belmont': 14, 'portia': 180, 'cato': 14, 'brutus': 502, 'temples': 13, 'nerissa': 65, 'seated': 14, 'pronounced': 14, 'leaps': 11, 'smith': 14, 'nephew': 31, 'casket': 17, 'accept': 32, 'morocco': 11, 'shylock': 106, 'pirates': 13, 'grudge': 10, 'tribe': 10, 'tubal': 18, 'launcelot': 80, 'gobbo': 27, 'rouse': 31, 'budge': 12, 'farthest': 10, 'doves': 14, 'defect': 11, 'allay': 13, 'jessica': 57, 'heinous': 19, 'torch': 35, 'fife': 12, 'gild': 13, 'placed': 29, 'curtains': 10, 'chooseth': 11, 'advantages': 10, 'scroll': 15, 'viii': 109, 'confused': 10, 'chiefest': 15, 'curtain': 19, 'worthless': 17, 'assume': 14, 'opens': 12, 'costly': 10, 'rebel': 27, 'rebels': 30, 'disgraced': 17, 'bleed': 42, 'coffin': 17, 'divided': 19, 'aloof': 20, 'bridegroom': 16, 'ornament': 14, 'skull': 16, 'sepulchre': 12, 'wisest': 14, 'eloquence': 11, 'demi': 10, 'exceed': 17, 'achieved': 14, 'salerio': 13, 'yon': 25, 'gaping': 11, 'balthasar': 39, 'endeavour': 14, 'padua': 38, 'enow': 10, 'coals': 10, 'moor': 79, 'bosoms': 27, 'tops': 11, 'offers': 19, 'clerk': 26, 'whet': 11, 'starved': 17, 'surgeon': 14, 'outlive': 15, 'rightful': 14, 'division': 21, 'contrived': 10, 'whereby': 19, 'thisbe': 25, 'dido': 13, 'willow': 25, 'footing': 16, 'stephano': 88, 'kneels': 24, 'bank': 17, 'orb': 10, 'spoils': 10, 'dim': 22, 'nightingale': 10, 'chanced': 12, 'athens': 67, 'theseus': 64, 'hippolyta': 24, 'philostrate': 13, 'athenian': 35, 'egeus': 15, 'hermia': 100, 'lysander': 102, 'demetrius': 164, 'sung': 22, 'examine': 12, 'mew': 12, 'jaws': 10, 'devour': 16, 'showers': 15, 'quince': 54, 'flute': 18, 'snout': 17, 'starveling': 13, 'duchess': 137, 'pyramus': 58, 'thisby': 29, 'nick': 10, 'wandering': 20, 'shriek': 10, 'rehearse': 11, 'puck': 48, 'coats': 22, 'oberon': 43, 'grove': 18, 'breathless': 13, 'lurk': 13, 'ale': 19, 'titania': 41, 'contagious': 11, 'lap': 20, 'dissension': 11, 'rear': 20, 'swim': 12, 'asleep': 59, 'spaniel': 10, 'hatred': 16, 'cowardice': 16, 'nymph': 10, 'spell': 11, 'triumphant': 12, 'purge': 21, 'peaseblossom': 11, 'cobweb': 12, 'mustardseed': 13, 'crew': 12, 'forsook': 11, 'rising': 23, 'centre': 12, 'hounds': 28, 'rules': 10, 'pleading': 12, 'scales': 11, 'footed': 10, 'canker': 23, 'streams': 12, 'recreant': 14, 'mounsieur': 11, 'dotage': 11, 'echo': 11, 'skies': 12, 'nymphs': 11, 'valentine': 225, 'transported': 10, 'vast': 16, 'player': 27, 'affright': 12, 'blade': 11, 'moans': 10, 'duck': 11, 'stabs': 22, 'gore': 11, 'tragedy': 13, 'hecate': 13, 'leonato': 175, 'beatrice': 159, 'messina': 10, 'pedro': 151, 'benedick': 201, 'pestilence': 14, 'allegiance': 26, 'lesson': 10, 'cherish': 26, 'bridge': 14, 'dreamt': 12, 'conrade': 31, 'borachio': 51, 'royally': 10, 'model': 15, 'overthrow': 17, 'brethren': 22, 'margaret': 309, 'ursula': 34, 'garland': 25, 'lieutenant': 57, 'alliance': 16, 'enraged': 17, 'gull': 10, 'trembling': 25, 'net': 11, 'proudly': 10, 'consume': 11, 'appearance': 12, 'ensuing': 11, 'dogberry': 59, 'verges': 27, 'watchman': 44, 'bench': 10, 'milan': 33, 'perfume': 11, 'arrant': 15, 'sexton': 18, 'appeared': 10, 'however': 12, 'antony': 515, 'underneath': 16, 'clamour': 19, 'pericles': 185, 'tyre': 29, 'gower': 68, 'antioch': 10, 'ashes': 20, 'antiochus': 30, 'mirror': 12, 'sights': 23, 'hills': 16, 'oppression': 15, 'untimely': 28, 'thaliard': 16, 'peaceful': 15, 'helicanus': 57, 'blast': 15, 'flatterer': 16, 'protector': 57, 'listening': 11, 'tarsus': 22, 'peers': 40, 'cleon': 40, 'dionyza': 32, 'famish': 18, 'towers': 12, 'cities': 15, 'craves': 25, 'trojan': 35, 'statue': 18, 'fisherman': 34, 'simonides': 56, 'thaisa': 49, 'que': 18, 'chivalry': 13, 'shouts': 14, 'pledge': 20, 'withhold': 10, 'capital': 13, 'chariot': 10, 'funeral': 24, 'nobles': 40, 'dissemble': 13, 'applaud': 12, 'lychorida': 18, 'thunders': 10, 'whistle': 11, 'midwife': 12, 'sailor': 23, 'cloudy': 15, 'corpse': 13, 'cerimon': 34, 'repose': 27, 'lift': 25, 'toss': 13, 'egyptian': 15, 'breathes': 13, 'marina': 102, 'leonine': 22, 'lasting': 17, 'blasted': 11, 'ladder': 20, 'boatswain': 21, 'mytilene': 14, 'boult': 54, 'quietly': 10, 'lysimachus': 50, 'household': 20, 'trimm': 11, 'barge': 12, 'island': 34, 'whispers': 20, 'perceived': 14, 'destruction': 30, 'alehouse': 10, 'heath': 10, 'huntsman': 16, 'servingman': 71, 'extremes': 16, 'christendom': 17, 'shrill': 12, 'adonis': 19, 'stephen': 13, 'henry': 1162, 'harms': 18, 'lucentio': 120, 'tranio': 140, 'pisa': 14, 'biondello': 74, 'baptista': 114, 'katharina': 124, 'bianca': 112, 'gremio': 91, 'hortensio': 116, 'gentler': 10, 'froward': 13, 'alarums': 28, 'plainness': 11, 'achieve': 13, 'plotted': 10, 'weighty': 14, 'petruchio': 210, 'grumio': 96, 'verona': 27, 'ldst': 10, 'lighted': 11, 'indifferent': 15, 'chafed': 10, 'insolence': 14, 'mantua': 23, 'survive': 10, 'halt': 10, 'lucrece': 45, 'est': 22, 'saddle': 11, 'ceremonies': 10, 'surly': 10, 'curtis': 28, 'nicholas': 11, 'walter': 33, 'gregory': 24, 'murdered': 12, 'asking': 12, 'witted': 10, 'alonso': 49, 'gonzalo': 67, 'cabin': 11, 'gape': 11, 'prospero': 143, 'miranda': 62, 'magic': 13, 'compassion': 11, 'perdition': 10, 'trash': 11, 'naples': 30, 'ariel': 90, 'caliban': 69, 'howl': 16, 'breasts': 12, 'confined': 18, 'control': 16, 'adrian': 14, 'stained': 11, 'rub': 12, 'dropp': 14, 'suggestion': 10, 'trinculo': 62, 'snare': 13, 'sinews': 23, 'prompt': 14, 'batter': 10, 'trespass': 22, 'joints': 21, 'ceres': 12, 'starts': 17, 'stout': 18, 'melting': 15, 'resign': 25, 'chains': 12, 'pointing': 12, 'cressida': 179, 'regal': 12, 'menelaus': 31, 'plains': 21, 'broils': 11, 'unarm': 13, 'cassandra': 23, 'tower': 82, 'andromache': 11, 'antenor': 15, 'helenus': 11, 'agamemnon': 99, 'sennet': 18, 'ulysses': 115, 'selfsame': 12, 'patroclus': 71, 'factious': 10, 'trojans': 14, 'allowance': 10, 'mouldy': 20, 'rape': 17, 'degenerate': 12, 'diomedes': 84, 'suffers': 10, 'warriors': 16, 'calchas': 14, 'diomed': 35, 'watchful': 11, 'pyrrhus': 13, 'devotion': 19, 'woful': 24, 'front': 23, 'wrathful': 13, 'excursions': 21, 'armies': 14, 'marching': 25, 'winchester': 67, 'orsino': 81, 'curio': 12, 'olivia': 153, 'purged': 10, 'viola': 137, 'illyria': 10, 'toby': 205, 'cesario': 18, 'malvolio': 130, 'madonna': 10, 'calamity': 10, 'sheriff': 26, 'roderigo': 105, 'returns': 17, 'fabian': 73, 'yeoman': 12, 'dieu': 10, 'vous': 27, 'votre': 10, 'map': 11, 'topas': 16, 'titus': 223, 'ingrateful': 12, 'staves': 11, 'proteus': 217, 'julia': 149, 'lucetta': 62, 'eglamour': 21, 'panthino': 21, 'drench': 10, 'silvia': 127, 'launce': 81, 'thurio': 71, 'closed': 10, 'dearth': 11, 'timeless': 10, 'rashness': 10, 'territories': 11, 'outlaw': 32, 'rewards': 11, 'heave': 13, 'laurence': 78, 'leontes': 146, 'camillo': 124, 'bohemia': 28, 'sicilia': 15, 'mamillius': 19, 'hermione': 61, 'polixenes': 80, 'drag': 10, 'churchyard': 13, 'antigonus': 28, 'cleomenes': 17, 'dion': 11, 'paulina': 85, 'emilia': 146, 'perdita': 39, 'chorus': 18, 'florizel': 57, 'autolycus': 75, 'mopsa': 17, 'dorcas': 14, 'rue': 18, 'tunes': 10, 'cyprus': 28, 'posterity': 13, 'london': 115, 'lancaster': 110, 'westmoreland': 100, 'channel': 13, 'christ': 10, 'mortimer': 72, 'glendower': 48, 'hotspur': 128, 'harry': 119, 'percy': 97, 'scot': 17, 'douglas': 59, 'angus': 11, 'northumberland': 152, 'plantagenet': 72, 'worcester': 62, 'jerusalem': 11, 'hal': 41, 'sblood': 12, 'zounds': 21, 'gadshill': 27, 'ned': 20, 'canterbury': 36, 'eastcheap': 10, 'peto': 23, 'scourge': 17, 'bolingbroke': 176, 'deposed': 22, 'edmund': 147, 'gage': 18, 'whipp': 11, 'york': 587, 'archbishop': 68, 'scroop': 27, 'chamberlain': 70, 'kent': 191, 'gaunt': 70, 'thursday': 17, 'stab': 17, 'jesu': 21, 'accompanied': 11, 'paul': 10, 'shrewsbury': 21, 'battles': 18, 'holland': 13, 'holes': 12, 'vernon': 44, 'coventry': 13, 'alban': 22, 'michael': 27, 'fickle': 10, 'haughty': 14, 'monmouth': 17, 'basely': 12, 'stafford': 21, 'insulting': 10, 'prophesy': 19, 'bishop': 77, 'pomfret': 17, 'hastings': 145, 'mowbray': 64, 'doll': 51, 'tearsheet': 40, 'commons': 32, 'arthur': 72, 'westminster': 16, 'surrey': 47, 'warwick': 420, 'bullcalf': 13, 'oxford': 55, 'bona': 22, 'norfolk': 127, 'sampson': 22, 'coronation': 21, 'hereford': 36, 'colevile': 16, 'clarence': 210, 'humphrey': 62, 'navy': 10, 'daggers': 13, 'davy': 45, 'ely': 30, 'bedford': 47, 'exeter': 95, 'lewis': 92, 'puissant': 11, 'dauphin': 107, 'cambridge': 18, 'victorious': 15, 'glories': 12, 'fluellen': 88, 'calais': 12, 'macmorris': 11, 'les': 22, 'moi': 10, 'bourbon': 11, 'montjoy': 20, 'burgundy': 81, 'chatillon': 15, 'rambures': 10, 'rouen': 14, 'suns': 11, 'erpingham': 11, 'leek': 13, 'bates': 10, 'williams': 31, 'faintly': 12, 'salisbury': 138, 'talbot': 181, 'suffolk': 255, 'heralds': 10, 'signal': 11, 'mayor': 55, 'reignier': 48, 'anjou': 17, 'weal': 16, 'eager': 10, 'pucelle': 76, 'cardinal': 299, 'conspirator': 14, 'pope': 19, 'beaufort': 13, 'auvergne': 15, 'somerset': 149, 'diadem': 13, 'discontented': 13, 'basset': 10, 'malicious': 12, 'lucy': 21, 'maine': 19, 'hew': 12, 'cromwell': 43, 'commanding': 10, 'legate': 10, 'dover': 17, 'angiers': 16, 'hautboys': 14, 'buckingham': 255, 'marquess': 10, 'eleanor': 12, 'hume': 20, 'horner': 10, 'simpcox': 22, 'stanley': 49, 'surveyor': 15, 'cade': 96, 'apothecary': 10, 'whitmore': 14, 'bevis': 12, 'clifford': 137, 'iden': 17, 'chop': 10, 'montague': 88, 'chancellor': 13, 'rutland': 25, 'xi': 27, 'elizabeth': 122, 'pembroke': 38, 'richmond': 69, 'wolsey': 106, 'secretary': 10, 'brandon': 11, 'lovell': 40, 'campeius': 22, 'gardiner': 32, 'griffith': 26, 'cranmer': 36, 'dorset': 27, 'butts': 10, 'elinor': 27, 'constance': 47, 'faulconbridge': 16, 'blanch': 22, 'stains': 10, 'pandulph': 29, 'hubert': 95, 'bigot': 15, 'melun': 10, 'aumerle': 70, 'bushy': 27, 'bagot': 18, 'derby': 35, 'ross': 70, 'willoughby': 14, 'flatterers': 11, 'berkeley': 11, 'carlisle': 14, 'fitzwater': 11, 'exton': 13, 'brakenbury': 26, 'catesby': 73, 'vaughan': 12, 'ratcliff': 39, 'tyrrel': 22, 'alexandria': 19, 'fulvia': 18, 'antonius': 10, 'charmian': 104, 'iras': 41, 'alexas': 34, 'domitius': 139, 'enobarbus': 152, 'octavius': 175, 'lepidus': 71, 'menas': 53, 'mardian': 18, 'ventidius': 22, 'mecaenas': 27, 'agrippa': 53, 'octavia': 42, 'philippi': 15, 'cassius': 229, 'marcellus': 47, 'marcus': 134, 'eros': 63, 'canidius': 18, 'scarus': 19, 'dolabella': 37, 'thyreus': 20, 'proculeius': 21, 'seleucus': 10, 'coriolanus': 212, 'marcius': 140, 'patricians': 19, 'menenius': 194, 'sicinius': 136, 'volsces': 22, 'cominius': 103, 'lartius': 38, 'tullus': 11, 'aufidius': 93, 'corioli': 22, 'volumnia': 67, 'virgilia': 35, 'valeria': 22, 'volsce': 12, 'publius': 33, 'quintus': 20, 'aedile': 18, 'hamlet': 471, 'denmark': 24, 'bernardo': 30, 'horatio': 158, 'norway': 14, 'fortinbras': 21, 'claudius': 131, 'gertrude': 95, 'polonius': 119, 'laertes': 106, 'ophelia': 87, 'reynaldo': 18, 'rosencrantz': 76, 'guildenstern': 64, 'osric': 31, 'calpurnia': 16, 'decius': 25, 'cicero': 15, 'casca': 72, 'titinius': 36, 'cinna': 38, 'metellus': 18, 'cimber': 17, 'trebonius': 16, 'ligarius': 14, 'lucilius': 62, 'pindarus': 19, 'messala': 46, 'varro': 36, 'strato': 13, 'volumnius': 12, 'clitus': 14, 'lear': 228, 'albany': 75, 'cornwall': 74, 'goneril': 82, 'regan': 104, 'cordelia': 62, 'edgar': 135, 'oswald': 55, 'nuncle': 17, 'macbeth': 287, 'duncan': 37, 'malcolm': 59, 'donalbain': 15, 'lennox': 34, 'banquo': 76, 'thane': 25, 'cawdor': 21, 'macduff': 107, 'fleance': 15, 'siward': 27, 'birnam': 11, 'dunsinane': 14, 'seyton': 12, 'othello': 332, 'iago': 359, 'cassio': 252, 'brabantio': 43, 'desdemona': 229, 'montano': 35, 'lodovico': 43, 'romeo': 311, 'capulet': 146, 'benvolio': 80, 'tybalt': 78, 'mercutio': 88, 'apemantus': 132, 'alcibiades': 61, 'lucullus': 15, 'caphis': 18, 'flaminius': 21, 'servilius': 20, 'timandra': 11, 'bandit': 12, 'andronicus': 232, 'saturninus': 70, 'bassianus': 46, 'lavinia': 77, 'martius': 19, 'mutius': 15, 'tamora': 89, 'chiron': 52, 'aaron': 97, 'saturnine': 16, 'goth': 12, 'aemilius': 13, 'collatine': 25}

#### break2

In [14]:
X = []

count = 0
for i in word_frequencies:
  #WE avoid words that has a frequency which is less than 10
  if word_frequencies[i] > 10:
    X = X + [count] * word_frequencies[i]
    count = count + 1

#### End input

In [17]:
B = len(set(X))
n = len(X)
print(n, B)

889996 321


In [ ]:
#Comparing top k elements

Eps = [0.1, 0.5, 1.0]
Trials = 10

B = len(set(X))
n = len(X)

k = 5
delta = 1e-6

#storing data
Dataset = []

#define max errors
Err0_max = []
Err1_max = []
Err2_max = []

#define min errors
Err0_avg = []
Err1_avg = []
Err2_avg = []

#Checking GPU
if torch.cuda.is_available():
    device = "cuda"
    print("GPU is available")
else:
    device = "cpu"
    print("GPU is unavailable")

for epsilon in Eps:

  #Tuning ST parameters
  p_st, tau_st = st_param(epsilon, delta, alpha = 1/6)
  print( f"ST parameters: p = {p_st}, tau = {tau_st}" )

  #Tuning UE parameters
  t1_ue, t0_ue = ue_param(epsilon)
  print( f"UE parameters: t1 = {t1_ue}, t0 = {t0_ue}" )

  #Tuning SRT parameters
  #eps_0 = 10
 # optional_p = 0.5
  tau_srt = 100 / epsilon
  t1 = 0.5
  p_srt, t0, ep0 = solve_system_SRT( n, tau_srt, epsilon/2, delta/2 )
  print(p_srt, t0, ep0)

  print( f"SRT parameters: t1 = {t1}, t0 = {t0}, p_srt = {p_srt}, tau = {tau_srt}, ep0 = {ep0}" )

  #define max errors
  Err0_max_temp = []
  Err1_max_temp = []
  Err2_max_temp = []

  #define min errors
  Err0_avg_temp = []
  Err1_avg_temp = []
  Err2_avg_temp = []

  for trials in range(Trials):

      #Creating random binomial samples
      X = binom.rvs(n = B, p=0.4, size=n)

      #Implementing ST
      cx0, phix0, calg0, phialg0 = SRT_v2( X, B, 0.0, 1.0, p_st, tau_st, device )
      print("ST works")

      #Implementing UE
      cx1, phix1, calg1, phialg1 = UE( X, B, t0_ue, t1_ue, 1.0, 0.0, device )
      print("UE works")

      #Implementing SRT
      #SRT_v2(X, B, t0, t1, ps, tau, device='cuda')
      cx2, phix2, calg2, phialg2 = SRT_v2( X, B, t0, t1, p_srt, tau_srt, device )
      print("SRT works")

      if trials == 0:
        for i in range(B):
          if cx0[i] != cx2[i]:
            print("Error: real data does not match.")

      #storing data
      Dataset.append( [cx0, calg0, calg1, calg2] )

      #Measuring errors
      err0 = []
      err1 = []
      err2 = []
      k_frequent = cx0.most_common(k)

      for i in range(len(k_frequent)):
          entry = k_frequent[i][0]
          err0.append( abs( cx0[entry] - calg0[entry] ) / n )
          err1.append( abs( cx0[entry] - calg1[entry] ) / n )
          err2.append( abs( cx0[entry] - calg2[entry] ) / n )

      if len(err0) == 0:
          err0.append(0)
      if len(err1) == 0:
          err1.append(0)
      if len(err2) == 0:
          err2.append(0)

      Err0_max_temp.append(max(err0))
      Err1_max_temp.append(max(err1))
      Err2_max_temp.append(max(err2))

      Err0_avg_temp.append(mean(err0))
      Err1_avg_temp.append(mean(err1))
      Err2_avg_temp.append(mean(err2))

      #print("checkpoint: trials =  ", trials)

  Err0_max.append( mean(Err0_max_temp) )
  Err1_max.append( mean(Err1_max_temp) )
  Err2_max.append( mean(Err2_max_temp) )

  Err0_avg.append( mean(Err0_avg_temp) )
  Err1_avg.append( mean(Err1_avg_temp) )
  Err2_avg.append( mean(Err2_avg_temp) )

  print("checkpoint: epsilon =  ", epsilon)

GPU is available
ST parameters: p = 0.01586043032734008, tau = 932.0055029135224
UE parameters: t1 = 0.5, t0 = 0.47502081252106
0.108285742267820 0.0521558887032313 2.89995295553475
SRT parameters: t1 = 0.5, t0 = 0.0521558887032313, p_srt = 0.108285742267820, tau = 1000.0, ep0 = 2.89995295553475


### Printing dataset

In [ ]:
print(Err0_max)
print(Err1_max)
print(Err2_max)
print(Err0_avg)
print(Err1_avg)
print(Err2_avg)
print(Dataset)

### Graphing

In [ ]:
plt.rcParams.update({'font.size': 16})

plt.figure(figsize=(10, 6))
plt.plot(Eps, Err0_max, '-o', label = "ST")
plt.plot(Eps, Err1_max, '-o', label = "UE")
plt.plot(Eps, Err2_max, '-o', label = "SRT")

plt.title(f"Shakespeare text")
plt.xlabel(f'$\epsilon$')
plt.ylabel('Max Error')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 16})

plt.figure(figsize=(10, 6))
plt.plot(Eps, Err0_avg, '-o', label = "ST")
plt.plot(Eps, Err1_avg, '-o', label = "UE")
plt.plot(Eps, Err2_avg, '-o', label = "SRT")

plt.title(f"Shakespeare text")
plt.xlabel(f'$\epsilon$')
plt.ylabel('Average Error')
plt.grid(True)
plt.legend()
plt.show()

### end input

In [ ]:
data_ptone = Dataset[0]
data_ptfive = Dataset[10]
data_one = Dataset[20]

In [ ]:
og_ptone = data_ptone[0]
st_ptone = data_ptone[1]
ue_ptone = data_ptone[2]
srt_ptone = data_ptone[3]

og_ptfive = data_ptfive[0]
st_ptfive = data_ptfive[1]
ue_ptfive = data_ptfive[2]
srt_ptfive = data_ptfive[3]

og_one = data_one[0]
st_one = data_one[1]
ue_one = data_one[2]
srt_one = data_one[3]

temp = {k: v for k, v in sorted(og_ptone.items(), key=lambda item: item[1], reverse = True)}
og_ptone = temp
temp = {k: v for k, v in sorted(og_ptfive.items(), key=lambda item: item[1], reverse = True)}
og_ptfive = temp
temp = {k: v for k, v in sorted(og_one.items(), key=lambda item: item[1], reverse = True)}
og_one = temp

new_st_ptone = []
new_st_ptfive = []
new_st_one = []

new_ue_ptone = []
new_ue_ptfive = []
new_ue_one = []

new_srt_ptone = []
new_srt_ptfive = []
new_srt_one = []

for k,v in og_ptone.items():
    new_st_ptone.append(st_pointone[k])
    new_ue_ptone.append(ue_pointone[k])
    new_srt_ptone.append(srt_pointone[k])

for k,v in og_ptfive.items():
    new_st_ptfive.append(st_one[k])
    new_ue_ptfive.append(ue_one[k])
    new_srt_ptfive.append(srt_one[k])

for k,v in og_one.items():
    new_st_one.append(st_one[k])
    new_ue_one.append(ue_one[k])
    new_srt_one.append(srt_one[k])

st_ptone = new_st_ptone
st_ptfive = new_st_ptfive
st_one = new_st_one

ue_ptone = new_ue_ptone
ue_ptfive = new_ue_ptfive
ue_one = new_ue_one

srt_ptone = new_srt_ptone
srt_ptfive = new_srt_ptfive
srt_one = new_srt_one

In [ ]:
import matplotlib.pyplot as plt


plt.rcParams.update({'font.size': 12})

#ep = 0.1

#processing data
ep = 0.1
cx0 = []
temp = list( og_ptone.values() )
cut = 5

for i in range(cut):
    cx0.append(temp[i])

calg0 = st_ptone[:cut]
calg1 = ue_ptone[:cut]
calg2 = srt_ptone[:cut]

#processing x-axis

words = list( word_frequencies.keys() )
indices = list(og_ptone.keys())[:cut]
x_axis_words = [words[i] for i in indices]

#making the plot
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharey=True, figsize=(12, 4))
f.suptitle(f"Shakespeare dataset, $\epsilon = {ep}$")
stepsize = max(1, len(calg0) // 4)

ax1.bar(range(len(calg0)), cx0,  color='b')
ax1.set_title("True histogram")
ax1.set_xticks(x_axis_words)
ax1.set_xlabel("item")
ax1.set_ylabel("count")

ax3.bar(range(len(calg0)), calg0,  color='b')
ax3.set_title(r"ST")
ax3.set_xticks(x_axis_words)
ax3.set_xlabel("item")
ax3.set_ylabel("count")

ax4.bar(range(len(calg1)), calg1,  color='b')
ax4.set_title(r"UE")
ax4.set_xticks(x_axis_words)
ax4.set_xlabel("item")
ax4.set_ylabel("count")

ax2.bar( range(len(calg2)), calg2,  color='b')
ax2.set_title(r"SRT")
ax2.set_xticks(x_axis_words)
ax2.set_xlabel("item")
ax2.set_ylabel("count")

plt.tight_layout(rect=[0, 0, 1.05, 1.05])

##################breaking########################

#ep = 0.5

#processing data
ep = 0.5
cx0 = []
temp = list( og_ptfive.values() )
cut = 5

for i in range(cut):
    cx0.append(temp[i])

calg0 = st_ptfive[:cut]
calg1 = ue_ptfive[:cut]
calg2 = srt_ptfive[:cut]

#processing x-axis

words = list( word_frequencies.keys() )
indices = list(og_ptfive.keys())[:cut]
x_axis_words = [words[i] for i in indices]

#making the plot

f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharey=True, figsize=(12, 4))
f.suptitle(f"Shakespeare dataset, $\epsilon = {ep}$")
stepsize = max(1, len(calg0) // 4)

ax1.bar(range(len(calg0)), cx0,  color='b')
ax1.set_title("True histogram")
ax1.set_xticks(x_axis_words)
ax1.set_xlabel("item")
ax1.set_ylabel("count")

ax3.bar(range(len(calg0)), calg0,  color='b')
ax3.set_title(r"ST")
ax3.set_xticks(x_axis_words)
ax3.set_xlabel("item")
ax3.set_ylabel("count")

ax4.bar(range(len(calg1)), calg1,  color='b')
ax4.set_title(r"UE")
ax4.set_xticks(x_axis_words)
ax4.set_xlabel("item")
ax4.set_ylabel("count")

ax2.bar( range(len(calg2)), calg2,  color='b')
ax2.set_title(r"SRT")
ax2.set_xticks(x_axis_words)
ax2.set_xlabel("item")
ax2.set_ylabel("count")

plt.tight_layout(rect=[0, 0, 1.05, 1.05])

##################breaking########################

#ep = 1.0

#processing data
ep = 1.0
cx0 = []
temp = list( og_one.values() )
cut = 5

for i in range(cut):
    cx0.append(temp[i])

calg0 = st_one[:cut]
calg1 = ue_one[:cut]
calg2 = srt_one[:cut]

#processing x-axis

words = list( word_frequencies.keys() )
indices = list(og_one.keys())[:cut]
x_axis_words = [words[i] for i in indices]

#making the plot

f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharey=True, figsize=(12, 4))
f.suptitle(f"Shakespeare dataset, $\epsilon = {ep}$")
stepsize = max(1, len(calg0) // 4)

ax1.bar(range(len(calg0)), cx0,  color='b')
ax1.set_title("True histogram")
ax1.set_xticks(x_axis_words)
ax1.set_xlabel("item")
ax1.set_ylabel("count")

ax3.bar(range(len(calg0)), calg0,  color='b')
ax3.set_title(r"ST")
ax3.set_xticks(x_axis_words)
ax3.set_xlabel("item")
ax3.set_ylabel("count")

ax4.bar(range(len(calg1)), calg1,  color='b')
ax4.set_title(r"UE")
ax4.set_xticks(x_axis_words)
ax4.set_xlabel("item")
ax4.set_ylabel("count")

ax2.bar( range(len(calg2)), calg2,  color='b')
ax2.set_title(r"SRT")
ax2.set_xticks(rx_axis_words)
ax2.set_xlabel("item")
ax2.set_ylabel("count")

plt.tight_layout(rect=[0, 0, 1.05, 1.05])